# ----- This is the SCRIPT for testing QUANTIZED MOBILENET after RETRAINING -----

# Including paths

In [1]:
#include
import argparse
import os, imp, sys
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from datetime import datetime
from ast import literal_eval
from torchvision.utils import save_image
from torch.autograd import Variable
import copy, math
import numpy as np


In [2]:
#my script
sys.path.append('../')
from data import get_dataset, get_num_classes
from preprocess import get_transform
from utils import *
import models
import quantization

# Load Network #

In [5]:
checkpoint_file = '/home/fariselli/Work/training-mixed-precision-quantized-networks/results-nips/mobilenet_224_0.75_quant_auto_fix/checkpoint.pth.tar'


In [6]:
if os.path.isdir(checkpoint_file):
#            results.load(os.path.join(checkpoint_file, 'results.csv'))
    checkpoint_file = os.path.join(
        checkpoint_file, 'model_best.pth.tar')
if os.path.isfile(checkpoint_file):
    checkpoint_loaded = torch.load(checkpoint_file)
    #checkpoint = checkpoint_loaded['state_dict']
    #model.load_state_dict(checkpoint, strict=False)
    print('Model pretrained' )
else:
    print('here')

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by access

AttributeError: Can't get attribute 'ScaledClippedLinearQuantizationChannelBias' on <module 'models.linear_quantized_modules' from '/home/fariselli/Work/training-mixed-precision-quantized-networks/models/linear_quantized_modules.py'>

In [ ]:
model_name = checkpoint_loaded['model']

In [8]:
model_config = checkpoint_loaded['config']
activ_bits = model_config['activ_bits']
activ_type = model_config['activ_type']
dataset = model_config['dataset']
input_dim = model_config['input_dim']
input_size = input_dim
num_classes = model_config['num_classes']
type_quant = model_config['type_quant']
weight_bits = model_config['weight_bits']
width_mult = model_config['width_mult']
additional_config = ''
quant_add_config = checkpoint_loaded['add_config']
fold_type = checkpoint_loaded['fold_type']

In [9]:
checkpoint_loaded['best_prec1']

66.846

# Load Model 

In [10]:
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))
model = models.__dict__[model_name]

nClasses = get_num_classes(dataset)
model_config = {'input_size': input_size, 'dataset': dataset, 'num_classes': nClasses, \
                'type_quant': type_quant, 'weight_bits': weight_bits, 'activ_bits': activ_bits,\
                'activ_type': activ_type, 'width_mult': width_mult, 'input_dim': input_size }



if additional_config is not '':
    model_config = dict(model_config, **literal_eval(additional_config))


In [11]:
model_config

{'activ_bits': 8,
 'activ_type': 'learned',
 'dataset': 'imagenet',
 'input_dim': 224.0,
 'input_size': 224.0,
 'num_classes': 1000,
 'type_quant': 'PerLayerSymPACT',
 'weight_bits': 8,
 'width_mult': 1.0}

In [12]:
model_state = checkpoint_loaded['state_dict']

In [13]:
quantizer_load = checkpoint_loaded['quantizer']

In [14]:
param_list = copy.deepcopy(quantizer_load.param_to_quantize)
for item in param_list:
    print(item['quant_type'], item['fold_type'])


PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights
PerLayerSymPACT folding_weights


In [15]:
for item in quantizer_load.param_to_quantize:
    print(item.keys() )

dict_keys(['quant_act', 'w_bits', 'name', 'batch_norm', 'act', 'quant_conv', 'conv', 'quant_type', 'bias', 'w_max_thr', 'w_clip', 'fold_type', 'bias_bits', 'weight', 'act_o_bits'])
dict_keys(['quant_act', 'w_bits', 'name', 'batch_norm', 'act', 'quant_conv', 'conv', 'quant_type', 'bias', 'w_max_thr', 'w_clip', 'fold_type', 'bias_bits', 'weight', 'act_o_bits'])
dict_keys(['quant_act', 'w_bits', 'name', 'batch_norm', 'act', 'quant_conv', 'conv', 'quant_type', 'bias', 'w_max_thr', 'w_clip', 'fold_type', 'bias_bits', 'weight', 'act_o_bits'])
dict_keys(['quant_act', 'w_bits', 'name', 'batch_norm', 'act', 'quant_conv', 'conv', 'quant_type', 'bias', 'w_max_thr', 'w_clip', 'fold_type', 'bias_bits', 'weight', 'act_o_bits'])
dict_keys(['quant_act', 'w_bits', 'name', 'batch_norm', 'act', 'quant_conv', 'conv', 'quant_type', 'bias', 'w_max_thr', 'w_clip', 'fold_type', 'bias_bits', 'weight', 'act_o_bits'])
dict_keys(['quant_act', 'w_bits', 'name', 'batch_norm', 'act', 'quant_conv', 'conv', 'quant_typ

In [16]:

quantizer_load.param_to_quantize[1]

{'act': LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
 tensor([4.4838], device='cuda:0', requires_grad=True), inplace),
 'act_o_bits': 8,
 'batch_norm': BatchNorm2d(32, eps=0.001, momentum=0.003, affine=True, track_running_stats=True),
 'bias': False,
 'bias_bits': 8,
 'conv': Conv2d_SAME(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False),
 'fold_type': 'folding_weights',
 'name': '1.0',
 'quant_act': ScaledClippedLinearQuantization(M0=0.6594409346580505, N0=-1, clip_val=255, inplace),
 'quant_conv': Conv2d_SAME(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32),
 'quant_type': 'PerLayerSymPACT',
 'w_bits': 8,
 'w_clip': None,
 'w_max_thr': Parameter containing:
 tensor(36.3637, device='cuda:0', requires_grad=True),
 'weight': tensor([[[[ 1.9302e+00,  1.3625e+00,  2.1403e+00],
           [ 4.5900e-01,  9.9429e-01,  1.5504e+00],
           [ 1.7504e+00,  6.5618e-01,  1.3216e+00]]],
 
 
         [[[ 3.3594e+0

# Load Model

In [16]:
model = models.__dict__[model_name]
model = model(**model_config)
if model is None :
    print('ERORRR')
    
#logging.info("created model with configuration: %s", model_config)
print(model)


dataset='imagenet',num_classes=1000,input_size=224.0
8 8 PerLayerSymPACT
This is a quantized Mobilenet with alpha=  1.0  input_size:  224.0  activation bits:  8  weight bits:  8  activation type:  learned


/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


fc.weight Error
fc.bias Error
Model Pretrained Loaded
mobilenet_quant_devel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d_SAME(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.003, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
      tensor([6.], requires_grad=True), inplace)
    )
    (1): Sequential(
      (0): Conv2d_SAME(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.003, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
      tensor([6.], requires_grad=True), inplace)
    )
    (2): Sequential(
      (0): Conv2d_SAME(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.003, affine=True, track_running_stats=True

In [17]:
quant_add_config

[]

In [18]:
quantizer = quantization.QuantOp(model, type_quant, weight_bits, batch_fold_delay=0,batch_fold_type=fold_type, act_bits=activ_bits, add_config=quant_add_config  )


Batch Folding:  False Batch Folding Delay:  0 Type folding_weights
[]
This is the quantized network: 
mobilenet_quant_devel(
  (model): Sequential(
    (0): Conv2d_SAME(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): ScaledClippedLinearQuantization(M0=1, N0=1, clip_val=255, inplace)
    (2): Conv2d_SAME(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (3): ScaledClippedLinearQuantization(M0=1, N0=1, clip_val=255, inplace)
    (4): Conv2d_SAME(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (5): ScaledClippedLinearQuantization(M0=1, N0=1, clip_val=255, inplace)
    (6): Conv2d_SAME(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    (7): ScaledClippedLinearQuantization(M0=1, N0=1, clip_val=255, inplace)
    (8): Conv2d_SAME(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (9): ScaledClippedLinearQuantization(M0=1, N0=1, clip_val=255, inplace)
    (10): Conv2d_SAME(

In [19]:
quantizer.batch_fold = True


# Load Dateset


In [20]:
import torchvision.transforms as transforms

normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

input_eval_transform = transforms.Compose([
        transforms.Scale(int(input_size) ),
        transforms.ToTensor(),
        normalize
    ])
input_transform = getattr(model, 'input_transform', input_eval_transform)

val_data = get_dataset(dataset, 'val', input_transform['eval'])
val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=32, shuffle=False,
    num_workers=8, pin_memory=True)


/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


# Load dataset and quantizer!!

In [21]:
for i,item in enumerate(quantizer_load.param_to_quantize):
    for key in item.keys():
        if key in ['bias_bits', 'quant_type','w_min_thr','min_clip', 'w_clip','w_max_thr','max_clip', 'fold_type','w_bits']:
            quantizer.param_to_quantize[i][key] = item[key]

In [22]:
quantizer.generate_deployment_model()

Layer:  0  | Sw:  tensor(0.0069)
[WEIGHT] Min:  -128.0 | Max:  127.0
Layer:  0  | Quant Bias Tensor - Error:  tensor(1062880.8750)
Layer PerLayerAsymPACT:  0  | M0:  tensor(0.5902) - n_exp:  -8
Layer PerLayerAsymPACT:  0  | Error:  tensor(0.)
Layer:  1  | Sw:  tensor(0.2852)
[WEIGHT] Min:  -112.0 | Max:  127.0
Layer:  1  | Quant Bias Tensor - Error:  tensor(4746.5449)
Layer PerLayerAsymPACT:  1  | M0:  tensor(0.5704) - n_exp:  -1
Layer PerLayerAsymPACT:  1  | Error:  tensor(0.)
Layer:  2  | Sw:  tensor(0.0101)
[WEIGHT] Min:  -128.0 | Max:  127.0
Layer:  2  | Quant Bias Tensor - Error:  tensor(973588.1250)
Layer PerLayerAsymPACT:  2  | M0:  tensor(0.6490) - n_exp:  -6
Layer PerLayerAsymPACT:  2  | Error:  tensor(0.)
Layer:  3  | Sw:  tensor(0.3635)
[WEIGHT] Min:  -128.0 | Max:  127.0
Layer:  3  | Quant Bias Tensor - Error:  tensor(4230.6436)
Layer PerLayerAsymPACT:  3  | M0:  tensor(0.7271) - n_exp:  -1
Layer PerLayerAsymPACT:  3  | Error:  tensor(0.)
Layer:  4  | Sw:  tensor(0.0078)
[W

In [23]:
model = torch.nn.DataParallel(model).cuda()


In [24]:
model.load_state_dict(model_state)


In [25]:
input_size

224.0

In [26]:
def forward(data_loader, model,  epoch=0, training=False, quantizer=None):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end = time.time()
    model.eval()
    model = model.cuda()
    
    max_i = 1
    min_i = -1
    bit_i = 8
    n_steps = (2**bit_i)-1
    eps = (max_i-min_i) / n_steps
    
    for i, (inputs, target) in enumerate(data_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        target = target.cuda(async=True)
        input_var = Variable(inputs.cuda(), volatile=not training)
        target_var = Variable(target)
        
        input_var = input_var.clamp(min_i,max_i).div(eps).round()
        
        if quantizer is not None:
            input_var = input_var.mul(eps)
            quantizer.store_and_quantize(training=False )

        # compute output
        output = model(input_var)
        if type(output) is list:
            output = output[0]
        values, indices = output.max(1)
        #print(indices, target)
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        top1.update(prec1[0], inputs.size(0))
        top5.update(prec5[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if quantizer is not None:
            quantizer.restore_real_value()

            
            
        if i % 100 == 0:
            print('{phase} - Epoch: [{0}][{1}/{2}]\t'
                         'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                         'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                         'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                         'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                             epoch, i, len(data_loader),
                             phase='TRAINING' if training else 'EVALUATING',
                             batch_time=batch_time,
                             data_time=data_time, top1=top1, top5=top5))

    print('Top1: ', top1.avg)
    return top1.avg, top5.avg




In [27]:
quantizer.generate_deployment_model()
quantizer.deployment_model.eval()
forward(val_loader, quantizer.deployment_model )

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


EVALUATING - Epoch: [0][0/1563]	Time 0.829 (0.829)	Data 0.772 (0.772)	Prec@1 87.500 (87.500)	Prec@5 96.875 (96.875)
EVALUATING - Epoch: [0][100/1563]	Time 0.014 (0.043)	Data 0.000 (0.029)	Prec@1 43.750 (61.541)	Prec@5 68.750 (82.704)
EVALUATING - Epoch: [0][200/1563]	Time 0.024 (0.048)	Data 0.000 (0.033)	Prec@1 62.500 (63.884)	Prec@5 84.375 (83.831)
EVALUATING - Epoch: [0][300/1563]	Time 0.028 (0.045)	Data 0.000 (0.028)	Prec@1 53.125 (63.715)	Prec@5 81.250 (84.956)
EVALUATING - Epoch: [0][400/1563]	Time 0.020 (0.042)	Data 0.000 (0.025)	Prec@1 43.750 (62.562)	Prec@5 78.125 (85.419)
EVALUATING - Epoch: [0][500/1563]	Time 0.017 (0.041)	Data 0.000 (0.024)	Prec@1 75.000 (62.625)	Prec@5 96.875 (85.778)
EVALUATING - Epoch: [0][600/1563]	Time 0.014 (0.041)	Data 0.000 (0.023)	Prec@1 78.125 (63.192)	Prec@5 87.500 (85.935)
EVALUATING - Epoch: [0][700/1563]	Time 0.027 (0.040)	Data 0.000 (0.023)	Prec@1 53.125 (61.992)	Prec@5 81.250 (84.950)
EVALUATING - Epoch: [0][800/1563]	Time 0.022 (0.040)	Data 

(tensor(56.9220, device='cuda:0'), tensor(80.3720, device='cuda:0'))

In [32]:
for i,item in enumerate(quantizer.deployment_model.modules() ):
    print(type(item))
    if type(item) in [models.linear_quantized_modules.Conv2d_SAME,torch.nn.modules.linear.Linear]:
        print('[WEIGHT] Min: ',item.weight.data.min().item(),'| Max: ',item.weight.data.max().item())
        print('[BIAS] Min: ',item.bias.data.min().item(),'| Max: ',item.bias.data.max().item())
    if type(item) in [models.linear_quantized_modules.ScaledClippedLinearQuantization]:
        print('[M0] ',item.M_ZERO,'| [N0]: ',item.N_ZERO)


<class 'models.imagenet.mobilenet.mobilenet_quant_devel'>
<class 'torch.nn.modules.container.Sequential'>
<class 'models.linear_quantized_modules.Conv2d_SAME'>
[WEIGHT] Min:  -128.0 | Max:  128.0
[BIAS] Min:  -128.0 | Max:  127.0
<class 'models.linear_quantized_modules.ScaledClippedLinearQuantization'>
[M0]  0.5902066230773926 | [N0]:  -8
<class 'models.linear_quantized_modules.Conv2d_SAME'>
[WEIGHT] Min:  -112.0 | Max:  128.0
[BIAS] Min:  -128.0 | Max:  127.0
<class 'models.linear_quantized_modules.ScaledClippedLinearQuantization'>
[M0]  0.5704110860824585 | [N0]:  -1
<class 'models.linear_quantized_modules.Conv2d_SAME'>
[WEIGHT] Min:  -128.0 | Max:  128.0
[BIAS] Min:  -128.0 | Max:  127.0
<class 'models.linear_quantized_modules.ScaledClippedLinearQuantization'>
[M0]  0.6489582657814026 | [N0]:  -6
<class 'models.linear_quantized_modules.Conv2d_SAME'>
[WEIGHT] Min:  -128.0 | Max:  128.0
[BIAS] Min:  -113.0 | Max:  127.0
<class 'models.linear_quantized_modules.ScaledClippedLinearQuanti

In [29]:
np.log2(2**31-1)

30.999999999328193

In [ ]:
#test trained model
forward(val_loader, quantizer_load.deployment_model )

In [33]:
model.module.model[1][1].running_mean

tensor([        nan,         nan,         nan,  2.3262e-43,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan, -2.3262e-43,         nan,         nan,  4.4491e+00,
                nan,         nan, -2.3262e-43,  2.3262e-43, -2.3262e-43,
         2.3262e-43,         nan,         nan, -2.3262e-43], device='cuda:0')

In [ ]:
quantizer.deployment_model.fc[0].bias.data.type()

In [ ]:
 model.module.fc.bias.data/quantizer.deployment_model.fc[0].bias.data

In [27]:
model.eval()
forward(val_loader, model,quantizer=quantizer )

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


EVALUATING - Epoch: [0][0/1563]	Time 10.679 (10.679)	Data 0.815 (0.815)	Prec@1 90.625 (90.625)	Prec@5 93.750 (93.750)
EVALUATING - Epoch: [0][100/1563]	Time 0.099 (0.308)	Data 0.001 (0.105)	Prec@1 9.375 (59.561)	Prec@5 68.750 (85.210)
EVALUATING - Epoch: [0][200/1563]	Time 0.143 (0.295)	Data 0.001 (0.142)	Prec@1 56.250 (59.142)	Prec@5 87.500 (84.593)


Process Process-9:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process Process-13:
Process Process-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-11:
Process Process-10:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK

  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.5/dist-packages/torchvision/datasets/folder.py", line 101, in __getitem__
    sample = self.loader(path)
  File "/usr/local/lib/python3.5/dist-packages/torchvision/datasets/folder.py", line 147, in default_loader
    return pil_loader(path)
  File "/usr/local/lib/python3.5/dist-packages/torchvision/datasets/folder.py", line 129, in pil_loader
    img = Image.open(f)
  File "/usr/local/lib/python3.5/dist-packages/PIL/Image.py", line 2539, in open
    prefix = fp.read(16)
KeyboardInterrupt
ERROR:root:Internal Pyth

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4e564a0c285f>", line 2, in <module>
    forward(val_loader, model,quantizer=quantizer )
  File "<ipython-input-25-07ea60390c3a>", line 18, in forward
    for i, (inputs, target) in enumerate(data_loader):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/usr/lib/python3.5/queue.py", line 164, in get
    self.not_empty.wait()
  File "/usr/lib/python3.5/threading.py", line 293, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/loc

KeyboardInterrupt: 

In [ ]:
weight_tensor = model.module.model[0][0].weight.data.clone().clamp(quantizer.param_to_quantize[0]['w_min_thr'].data,quantizer.param_to_quantize[0]['w_max_thr'].data)
w_min,w_max = weight_tensor.min().item(),weight_tensor.max().item()
nlevels = ((2**8)-1)
Sw= (w_max-w_min)/nlevels
Sw

In [35]:
quantizer.store_and_quantize(training=False )



In [ ]:
quantizer_load.deployment_model.model[0].weight.data.mul(Sw)

In [ ]:
model.module.model[0][0].weight.data

In [34]:
quantizer.restore_real_value()


In [28]:
quantizer.param_to_quantize[27]['batch_norm'] is not None

False

# --------------

# Parameters

In [ ]:
results_dir = './results' #results dir
save = 'imagenet/mobilenet_test_8bit' #saved folder
dataset = 'imagenet' #dataset name or folder
model_name = 'mobilenet' #model architecture

input_size = 224 #image input size
additional_config = str() #additional architecture configuration (need to be string str() )
ttype = 'torch.cuda.FloatTensor' #type of tensor - e.g torch.cuda.HalfTensor
gpus = '0' #gpus used for training - e.g 0,1,3
workers = 8 #number of data loading workers
epochs = 60 # number of total epochs to run
start_epoch = 0 # manual epoch number (useful on restarts)
batch_size = 1 # mini-batch size 
optimizer = 'SGD' # optimizer function used
lr = 0.1 # initial learning rate
momentum = 0.9
weight_decay = 1e-4 #weight decay
print_freq = 10 # print frequency
resume = '' #path to latest checkpoint
evaluate = False # evaluate model FILE on validation set
save_check = False # saving the checkpoint

# Setup GPU devices

In [ ]:
if 'cuda' in ttype:
    gpus = [int(i) for i in gpus.split(',')]
    print('Selected GPUs: ', gpus)
    torch.cuda.set_device(gpus[0])
    cudnn.benchmark = True
else:
    gpus = None


# Load Pretrained Real model

In [6]:
type_quant = 'LinearAsymmetricLayer' #type of binarization process
#type_quant = None
weight_bits = 4
activ_bits = 4
activ_type = 'learned' 
i_dim = 224
width_mult = 1

In [7]:
model = models.__dict__[model_name]
nClasses = get_num_classes(dataset)
model_config = {'input_size': input_size, 'dataset': dataset, 'num_classes': nClasses, \
                'type_quant': type_quant, 'weight_bits': weight_bits, 'activ_bits': activ_bits,\
                'activ_type': activ_type, 'width_mult': width_mult, 'input_dim': i_dim }

if additional_config is not '':
    model_config = dict(model_config, **literal_eval(additional_config))

model = model(**model_config)
if model is None :
    print('ERORRR')
    
#logging.info("created model with configuration: %s", model_config)
print(model)



input_size=224,num_classes=1000,dataset='imagenet'
4 4 LinearAsymmetricLayer
This is a quantized Mobilenet with alpha=  1  input_size:  224  activation bits:  4  weight bits:  4  activation type:  learned
mobilenet_quant_devel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=4, clip_val=Parameter containing:
      tensor([6.], requires_grad=True), inplace)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=4, clip_val=Parameter containing:
      tensor([6.], requires_grad=True), inplace)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel

# GPU Type

In [8]:
model.type(ttype)


mobilenet_quant_devel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=4, clip_val=Parameter containing:
      tensor([6.], device='cuda:0', requires_grad=True), inplace)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=4, clip_val=Parameter containing:
      tensor([6.], device='cuda:0', requires_grad=True), inplace)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantiza

# Loss function

In [9]:
criterion = nn.CrossEntropyLoss()
criterion.type(ttype)

CrossEntropyLoss()

# Dataset and dataloader

In [10]:
default_transform = {
    'train': get_transform(dataset,
                           input_size=input_size, augment=True),
    'eval': get_transform(dataset,
                          input_size=input_size, augment=False)
}
transform = getattr(model, 'input_transform', default_transform)
regime = getattr(model, 'regime', {0: {'optimizer': optimizer,
                                       'lr': lr,
                                       'momentum': momentum,
                                       'weight_decay': weight_decay}})



/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")
/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [11]:
val_data = get_dataset(dataset, 'val', transform['eval'])
val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True)


train_data = get_dataset(dataset, 'train', transform['train'])
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size, shuffle=True,
    num_workers=workers, pin_memory=True)


# Optimizer

In [12]:
params_dict = dict(model.named_parameters())
params = []
for key, value in params_dict.items():
    if 'clip_val' in key:
        params += [{'params':value,'weight_decay': 5e-4}]
    else:
        params += [{'params':value}]
optimizer = torch.optim.SGD(params, lr=0.1)

In [13]:
optimizer = adjust_optimizer(optimizer, 0, regime)

In [14]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    dampening: 0
    eps: 1e-08
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    dampening: 0
    eps: 1e-08
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    dampening: 0
    eps: 1e-08
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 3
    amsgrad: False
    betas: (0.9, 0.999)
    dampening: 0
    eps: 1e-08
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 4
    amsgrad: False
    betas: (0.9, 0.999)
    dampening: 0
    eps: 1e-08
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0.0005

Parameter Group 5
    amsgrad: False
    betas: (0.9, 0.999)
    dampening: 0
    eps: 1e-08
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Para

# Quantizer

In [15]:
add_config = [{'layer':0,'a_bits':4,'w_bits':8},{'layer':5,'a_bits':5,'w_bits':8}]

In [40]:
test_str = '[{"layer":0,"a_bits":4,"w_bits":8},{"layer":5,"a_bits":5,"w_bits":8}]'
print(type(test_str))

<class 'str'>


In [41]:
import json

In [42]:
my_dict = json.loads(test_str)
print (my_dict)


[{'layer': 0, 'a_bits': 4, 'w_bits': 8}, {'layer': 5, 'a_bits': 5, 'w_bits': 8}]


In [16]:
quantizer = quantization.QuantOp(model, type_quant, weight_bits, batch_fold_delay=0, act_bits=activ_bits,add_config=add_config  )

quantizer.deployment_model.type(ttype)
quantizer.add_params_to_optimizer(optimizer)

Batch Folding:  False Batch Folding Delay:  0 Type folding_thresh
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymmetricLayer
Tipe of Quantization:  LinearAsymm

# Pretrain

In [24]:
quantizer.init_parameters()

In [15]:
for i,v in enumerate(checkpoint):
    print(i,v)

0 module.model.0.0.weight
1 module.model.0.1.weight
2 module.model.0.1.bias
3 module.model.0.1.running_mean
4 module.model.0.1.running_var
5 module.model.0.1.num_batches_tracked
6 module.model.1.0.weight
7 module.model.1.1.weight
8 module.model.1.1.bias
9 module.model.1.1.running_mean
10 module.model.1.1.running_var
11 module.model.1.1.num_batches_tracked
12 module.model.2.0.weight
13 module.model.2.1.weight
14 module.model.2.1.bias
15 module.model.2.1.running_mean
16 module.model.2.1.running_var
17 module.model.2.1.num_batches_tracked
18 module.model.3.0.weight
19 module.model.3.1.weight
20 module.model.3.1.bias
21 module.model.3.1.running_mean
22 module.model.3.1.running_var
23 module.model.3.1.num_batches_tracked
24 module.model.4.0.weight
25 module.model.4.1.weight
26 module.model.4.1.bias
27 module.model.4.1.running_mean
28 module.model.4.1.running_var
29 module.model.4.1.num_batches_tracked
30 module.model.5.0.weight
31 module.model.5.1.weight
32 module.model.5.1.bias
33 module.m

# -----   End Initialization --------------------- #

torch.Size([32, 1, 3, 3])

In [20]:
weight_tensor = model.module.model[1][0].weight.data.clone()
batch_layer = model.module.model[1][1]
weight_tensor_size = model.module.model[1][0].weight.data.size()
v0,v1,v2,v3 = weight_tensor_size
eps = batch_layer.eps
gamma_tensor = batch_layer.weight.data 
beta_tensor = batch_layer.bias.data 
mu_tensor = batch_layer.running_mean
var_tensor = batch_layer.running_var

#assuming convolution bias == False
bias_tensor = mu_tensor.clone() 
bias_tensor = bias_tensor.mul(-1)

#folded weight
#var_tensor[var_tensor.le(0)] = 0       #correction to prevent case sigma >= 0
if var_tensor.le(0).sum() >0:
    print('erorr')
sigma_tensor = var_tensor.add(eps).sqrt()
gamma_over_sigma = gamma_tensor / sigma_tensor
gamma_over_sigma_tensor = gamma_over_sigma.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(v0,v1,v2,v3 )
weight_tensor = weight_tensor * gamma_over_sigma_tensor
bias_tensor = (bias_tensor*gamma_over_sigma)+beta_tensor


In [23]:
gamma_over_sigma

tensor([ 2.2468,  1.6286,  2.2804, -1.0486, 11.8503,  1.6791,  1.5009, -0.4797,
         1.7677,  1.2378,  0.9371,  0.0826,  0.1209, -0.3763,  5.4261, 12.1442,
        -0.0959,  1.0655,  1.7398,  8.4671,  2.4145,  1.1532,  9.0097,  4.7418,
         0.9083,  0.6108,  1.9244, -2.9012,  1.7007,  0.9227,  0.9232,  0.7745],
       device='cuda:0')

In [25]:
gamma_over_sigma_tensor


tensor([[[[ 2.2468,  2.2468,  2.2468],
          [ 2.2468,  2.2468,  2.2468],
          [ 2.2468,  2.2468,  2.2468]]],


        [[[ 1.6286,  1.6286,  1.6286],
          [ 1.6286,  1.6286,  1.6286],
          [ 1.6286,  1.6286,  1.6286]]],


        [[[ 2.2804,  2.2804,  2.2804],
          [ 2.2804,  2.2804,  2.2804],
          [ 2.2804,  2.2804,  2.2804]]],


        [[[-1.0486, -1.0486, -1.0486],
          [-1.0486, -1.0486, -1.0486],
          [-1.0486, -1.0486, -1.0486]]],


        [[[11.8503, 11.8503, 11.8503],
          [11.8503, 11.8503, 11.8503],
          [11.8503, 11.8503, 11.8503]]],


        [[[ 1.6791,  1.6791,  1.6791],
          [ 1.6791,  1.6791,  1.6791],
          [ 1.6791,  1.6791,  1.6791]]],


        [[[ 1.5009,  1.5009,  1.5009],
          [ 1.5009,  1.5009,  1.5009],
          [ 1.5009,  1.5009,  1.5009]]],


        [[[-0.4797, -0.4797, -0.4797],
          [-0.4797, -0.4797, -0.4797],
          [-0.4797, -0.4797, -0.4797]]],


        [[[ 1.7677,  1.7677,  1.

# Evaluate

In [101]:
net = nn.BatchNorm2d(5,momentum=1)

In [116]:
net.train()

BatchNorm2d(5, eps=1e-05, momentum=1, affine=True)

In [118]:
torch.__version__

'0.3.0.post4'

In [114]:
net.eval()

BatchNorm2d(5, eps=1e-05, momentum=1, affine=True)

In [104]:
gamma=net.weight.data
gamma


 0.9503
 0.9339
 0.7016
 0.6154
 0.1521
[torch.FloatTensor of size 5]

In [105]:
beta=net.bias.data

In [117]:
mean=net.running_mean
mean


 0.3295
 0.3252
 0.2393
 0.3410
 0.5642
[torch.FloatTensor of size 5]

In [107]:
var=net.running_var

In [1]:
sigma = var.add(net.eps).sqrt()
sigma

NameError: name 'var' is not defined

In [109]:
gamma_over_sigma = gamma / sigma
gamma_over_sigma


 3.5944
 2.8608
 2.3078
 2.3921
 0.3541
[torch.FloatTensor of size 5]

In [110]:
((x.data[0]-mean)*gamma_over_sigma)+beta


-0.8459
-0.4420
-0.3899
 0.5558
 0.1298
[torch.FloatTensor of size 5]

In [59]:
x = Variable(torch.rand(4,5))
x

Variable containing:
 0.0941  0.1706  0.0703  0.5733  0.9306
 0.2950  0.1742  0.1702  0.1021  0.0853
 0.2228  0.1414  0.0300  0.1352  0.9231
 0.7059  0.8143  0.6866  0.5534  0.3178
[torch.FloatTensor of size 4x5]

In [103]:
y = net(x)
y

Variable containing:
-0.9767 -0.5104 -0.4502  0.6417  0.1498
-0.1429 -0.4987 -0.1842 -0.6599 -0.1958
-0.4428 -0.6068 -0.5577 -0.5684  0.1467
 1.5624  1.6159  1.1921  0.5865 -0.1008
[torch.FloatTensor of size 4x5]

# Try single batch

In [25]:
training = False
model.eval()
quantizer.apply_graph_transforms(0)
quantizer.generate_deployment_model()

Frozen Batch Normalization Layers statistics!
Layer:  0
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  1
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  2
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  3
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  4
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  5
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  6
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  7
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  8
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  9
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  10
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  11
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  12
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  13
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  14
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  15
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  16
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  17
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  18
A bits:  4  | a_min:  0  | a_max:  6.0
Layer:  19
A b

In [26]:
#single input
inputs = torch.randn(1,3,32,32)
input_var = Variable(inputs.type(ttype))
quantizer.store_and_quantize(training=training)
output_1 = model(input_var)
quantizer.restore_real_value()            
output_2 = quantizer.deployment_model(input_var)
values_1, indices_1 = output_1.max(1)
values_2, indices_2 = output_2.max(1)



In [28]:
nn.Sequential(model.module.model[0],model.module.model[1])(input_var)

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [29]:
first = nn.Sequential(quantizer.deployment_model.model[0],quantizer.deployment_model.model[1],quantizer.deployment_model.model[2],quantizer.deployment_model.model[3])
first(input_var)
#quantizer.deployment_model.model[0](input_var)

tensor([[[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         ...,

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  

In [38]:
quantizer.deployment_model.fc[0].bias

Parameter containing:
tensor([ 0.0269, -0.6061,  0.3412,  0.7959,  0.3926, -0.1135, -0.1445,  0.3798,
        -0.5357, -0.5702], device='cuda:0')

In [27]:
# compare fake-quantized validation and inference
N = 10000
error = 0
for i in range(N):
    #i, (inputs, target) = next(enumerate(val_loader))
    inputs = torch.randn(1,3,32,32)
    input_var = Variable(inputs.type(ttype))
    quantizer.store_and_quantize(training=training)
    output_1 = model(input_var)
    quantizer.restore_real_value()            
    output_2 = quantizer.deployment_model(input_var)
    values_1, indices_1 = output_1.max(1)
    values_2, indices_2 = output_2.max(1)
    if indices_1[0].item() == indices_2[0].item():
        pass
    else:
        print('Different!!!! ', values_1, values_2)
        print(i)
        error += 1

if error == 0:
    print('check OKKKK!!!!')
else:
    print('number of errors', error)

Different!!!!  tensor([7.3045], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
0
Different!!!!  tensor([5.6645], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
1
Different!!!!  tensor([6.4845], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
2
Different!!!!  tensor([5.2310], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3
Different!!!!  tensor([4.7786], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
4
Different!!!!  tensor([6.7956], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
5
Different!!!!  tensor([6.9464], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
6
Different!!!!  tensor([7.2668], device='cuda:0', grad_fn=<MaxB

Different!!!!  tensor([5.0707], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
62
Different!!!!  tensor([8.3790], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
63
Different!!!!  tensor([7.5496], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
64
Different!!!!  tensor([8.3130], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
65
Different!!!!  tensor([5.3441], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
66
Different!!!!  tensor([5.4101], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
67
Different!!!!  tensor([5.7965], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
68
Different!!!!  tensor([6.3149], device='cuda:0', grad_f

Different!!!!  tensor([6.9652], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
123
Different!!!!  tensor([5.6268], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
124
Different!!!!  tensor([7.4365], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
125
Different!!!!  tensor([6.0981], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
126
Different!!!!  tensor([5.8719], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
127
Different!!!!  tensor([4.6089], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
128
Different!!!!  tensor([6.7767], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
129
Different!!!!  tensor([5.3629], device='cuda:0',

Different!!!!  tensor([5.0330], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
186
Different!!!!  tensor([5.0802], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
187
Different!!!!  tensor([5.9190], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
188
Different!!!!  tensor([6.1358], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
189
Different!!!!  tensor([5.2970], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
190
Different!!!!  tensor([5.6080], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
191
Different!!!!  tensor([5.2592], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
192
Different!!!!  tensor([6.1169], device='cuda:0',

Different!!!!  tensor([6.9652], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
250
Different!!!!  tensor([5.4289], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
251
Different!!!!  tensor([6.1641], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
252
Different!!!!  tensor([8.3224], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
253
Different!!!!  tensor([9.2367], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
254
Different!!!!  tensor([5.7399], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
255
Different!!!!  tensor([4.9011], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
256
Different!!!!  tensor([5.6457], device='cuda:0',

...

Different!!!!  tensor([7.3705], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3471
Different!!!!  tensor([5.1838], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3472
Different!!!!  tensor([4.9011], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3473
Different!!!!  tensor([4.5618], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3474
Different!!!!  tensor([3.9303], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3475
Different!!!!  tensor([6.8427], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3476
Different!!!!  tensor([5.5891], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3477
Different!!!!  tensor([6.9087], device='c

Different!!!!  tensor([6.6542], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3534
Different!!!!  tensor([4.8822], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3535
Different!!!!  tensor([5.4289], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3536
Different!!!!  tensor([5.4855], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3537
Different!!!!  tensor([6.1358], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3538
Different!!!!  tensor([7.9172], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3539
Different!!!!  tensor([5.3441], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3540
Different!!!!  tensor([5.0519], device='c

Different!!!!  tensor([6.9652], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3597
Different!!!!  tensor([6.1452], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3598
Different!!!!  tensor([5.3629], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3599
Different!!!!  tensor([5.4006], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3600
Different!!!!  tensor([5.1367], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3601
Different!!!!  tensor([5.8907], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3602
Different!!!!  tensor([5.0802], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3603
Different!!!!  tensor([7.3045], device='c

Different!!!!  tensor([6.2772], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3659
Different!!!!  tensor([5.4006], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3660
Different!!!!  tensor([6.6165], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3661
Different!!!!  tensor([5.3818], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3662
Different!!!!  tensor([4.5524], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3663
Different!!!!  tensor([5.5326], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3664
Different!!!!  tensor([6.7579], device='cuda:0', grad_fn=<MaxBackward0>) tensor([35.7500], device='cuda:0', grad_fn=<MaxBackward0>)
3665
Different!!!!  tensor([3.7418], device='c

KeyboardInterrupt: 

In [14]:
x = torch.randn(3,2)

In [28]:
(x > 0)| (x > -0.5)

tensor([[1, 0],
        [1, 1],
        [1, 1]], dtype=torch.uint8)

In [30]:
x.masked_fill_(x>0.5, 0)

tensor([[ 0.0000, -2.1960],
        [ 0.4333,  0.0000],
        [ 0.3283,  0.3488]])

In [31]:
x

tensor([[ 0.0000, -2.1960],
        [ 0.4333,  0.0000],
        [ 0.3283,  0.3488]])

In [16]:
model

mobilenet_quant_devel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ClippedLinearQuantization(num_bits=8, clip_val=6)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ClippedLinearQuantization(num_bits=8, clip_val=6)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ClippedLinearQuantization(num_bits=8, clip_val=6)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [26]:
print(indices_1, indices_2)
print(output_1)
print(output_2)


tensor([2], device='cuda:0') tensor([5], device='cuda:0')
tensor([[ 1.0216,  1.0125,  2.6960,  1.9408, -1.0364,  2.6033, -0.5207, -0.8645,
         -5.0692, -1.7818]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([[  7290.0000,   7771.5000,  20460.5000,  15013.5000,  -7461.7500,
          20608.2500,  -4499.2500,  -6394.0000, -39368.0000, -13401.0000]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)


In [21]:
quantizer.store_and_quantize(training=training)
bias_rr = model.module.fc.bias.data.clone()
oo1 = model(input_var)- bias_rr
quantizer.restore_real_value()
oo2 = quantizer.deployment_model(input_var)  - quantizer.deployment_model.fc[0].bias.data


In [22]:
oo_values_1, oo_indices_1 = oo1.max(1)
oo_values_2, oo_indices_2 = oo2.max(1)
print(oo_indices_1, oo_indices_2)
print(oo1)
print(oo2)
print(bias_rr)

tensor([1], device='cuda:0') tensor([1], device='cuda:0')
tensor([[ 1.6300,  3.8400,  2.8988, -1.6477, -1.8950, -0.0602,  2.2296, -1.1928,
         -3.9970, -1.8142]], device='cuda:0', grad_fn=<ThSubBackward>)
tensor([[ 12508.5000,  29883.5000,  21955.5000, -12784.5000, -13882.0000,
           -348.2500,  16404.2500,  -8747.5000, -31339.2500, -13704.2500]],
       device='cuda:0', grad_fn=<ThSubBackward>)
tensor([ 0.0271, -0.6058,  0.3413,  0.7958,  0.3924, -0.1132, -0.1445,  0.3799,
        -0.5359, -0.5703], device='cuda:0')


In [24]:
oo_values_1, oo_indices_1 = oo1.max(1)
oo_values_2, oo_indices_2 = oo2.max(1)
print(oo_indices_1, oo_indices_2)
print(oo1)
print(oo2)
print(bias_rr)

tensor([1], device='cuda:0') tensor([1], device='cuda:0')
tensor([[ 1.6300,  3.8400,  2.8988, -1.6477, -1.8950, -0.0602,  2.2296, -1.1928,
         -3.9970, -1.8142]], device='cuda:0', grad_fn=<ThSubBackward>)
tensor([[ 12508.5000,  29883.5000,  21955.5000, -12784.5000, -13882.0000,
           -348.2500,  16404.2500,  -8747.5000, -31339.2500, -13704.2500]],
       device='cuda:0', grad_fn=<ThSubBackward>)
tensor([ 0.0271, -0.6058,  0.3413,  0.7958,  0.3924, -0.1132, -0.1445,  0.3799,
        -0.5359, -0.5703], device='cuda:0')


In [23]:
quantizer.deployment_model.fc[0].bias.data

tensor([  207., -4646.,  2615.,  6101.,  3010.,  -870., -1108.,  2911., -4107.,
        -4371.], device='cuda:0')

In [24]:
output_1

tensor([[ 1.7513, -1.5044,  2.8504,  2.2066, -1.1107, -0.1140,  2.7132, -0.2327,
         -3.6545, -2.9618]], device='cuda:0', grad_fn=<ThAddmmBackward>)

In [25]:
output_2

tensor([[ 12914.2500, -10423.2500,  21100.5000,  16114.0000,  -8682.7500,
           -955.2500,  21532.7500,  -1240.7500, -28277.2500, -22523.2500]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)

In [26]:
model.module.fc.bias.data

tensor([ 0.0269, -0.6061,  0.3412,  0.7959,  0.3926, -0.1135, -0.1445,  0.3798,
        -0.5357, -0.5702], device='cuda:0')

In [27]:
quantizer.deployment_model.fc[0].bias.data

tensor([  207., -4646.,  2615.,  6101.,  3010.,  -870., -1108.,  2911., -4107.,
        -4371.], device='cuda:0')

In [90]:
for i_l,layer in enumerate(quantizer.param_to_quantize):
    print(layer['act'])

ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)
ClippedLinearQuantization(num_bits=8, clip_val=6)


In [84]:
model.module.fc.bias

Parameter containing:
tensor([ 0.0269, -0.6061,  0.3412,  0.7959,  0.3926, -0.1135, -0.1445,  0.3798,
        -0.5357, -0.5702], device='cuda:0', requires_grad=True)

In [68]:
quantizer.deployment_model

mobilenet_quant_devel(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ScaledClippedLinearQuantization(M=0.1803930699825287, clip_val=255)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
    (3): ScaledClippedLinearQuantization(M=2.054954767227173, clip_val=255)
    (4): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (5): ScaledClippedLinearQuantization(M=0.008188934065401554, clip_val=255)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
    (7): ScaledClippedLinearQuantization(M=0.7202885746955872, clip_val=255)
    (8): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (9): ScaledClippedLinearQuantization(M=0.004449482541531324, clip_val=255)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
    (11): ScaledClippedLinearQuantization(M=0.5195009112358093, clip_val=255)
    (12): Conv2d(128, 128, kernel

In [36]:
output_1.size()

torch.Size([49, 10])

In [33]:
innn = torch.randn(1,3,224,224)
innn

tensor([[[[-1.6592e+00, -9.9213e-01, -1.8538e+00,  ..., -1.6261e+00,
            1.5950e+00, -1.5785e+00],
          [ 9.5142e-01,  1.1928e+00,  2.8836e+00,  ..., -7.2053e-01,
           -7.4531e-02,  3.7682e-01],
          [-3.1280e-01,  2.1150e+00, -1.7433e-01,  ...,  7.0935e-01,
           -1.6594e+00,  2.1062e-01],
          ...,
          [ 3.3497e-02,  6.2067e-01,  6.3820e-01,  ...,  4.8994e-01,
            2.4570e-01,  6.8401e-01],
          [ 1.2982e-01, -1.5967e+00, -3.2398e-01,  ...,  6.2351e-01,
            4.4640e-01, -7.6585e-01],
          [ 3.9689e-01, -7.4495e-01, -1.2975e+00,  ...,  2.0225e+00,
            2.7354e-01, -6.5239e-01]],

         [[-3.9841e-02,  7.7911e-01,  1.1855e+00,  ..., -6.1800e-01,
            5.0772e-01,  4.5049e-01],
          [ 1.1449e+00,  1.2373e+00, -3.3193e-01,  ...,  8.9476e-01,
            7.7673e-01,  2.1901e+00],
          [-5.0232e-01, -3.0888e-01,  5.3224e-01,  ...,  7.4330e-01,
            8.5863e-02, -9.9452e-01],
          ...,
     

In [24]:
# forward and backward
for _ in range(100):
    i, (inputs, target) = next(enumerate(val_loader))
    if gpus is not None:
        target = target.cuda(async=True)
    input_var = Variable(inputs.type(ttype), volatile=not training)
    target_var = Variable(target)
    quantizer.store_and_quantize(get_quant_params=not training)
    output = model(input_var)
    loss = criterion(output, target_var)
    print(loss.data)
    #print(output.data)
    #print(model.model[0][1].weight.data)
    optimizer.zero_grad()
    loss.backward()
    quantizer.restore_real_value()            
    quantizer.backprop_quant_gradients()  
    optimizer.step()
    #print(model.model[0][1].weight.data)

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1a90929208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 345, in get
    return ForkingPickler.loads(res)
  File "/usr/local/lib/python3.5/dist-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/usr/lib/python3.5/multiprocessing/reduction.py", line 181, in recv_handle
    return recvfds(s, 1)[0]
  File "/usr/lib/python3.5/multiprocessing/reduction.py", line 152, in rec

TypeError: store_and_quantize() got an unexpected keyword argument 'update_float_params'

In [ ]:
model.model[0][1].running_var

In [24]:
model.eval()

DataParallel(
  (module): mobilenet_quant_devel(
    (model): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ClippedLinearQuantization(num_bits=8, clip_val=6)
      )
      (1): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ClippedLinearQuantization(num_bits=8, clip_val=6)
      )
      (2): Sequential(
        (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ClippedLinearQuantization(num_bits=8, clip_val=6)
      )
      (3): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 

In [15]:
training=False

In [18]:
i, (inputs, target) = next(enumerate(val_loader))


OSError: [Errno 12] Cannot allocate memory

In [21]:
if gpus is not None:
    target = target.cuda(async=True)
input_var = Variable(inputs.type(ttype), volatile=not training)
target_var = Variable(target)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
input_var = Variable(torch.randn(1,3,32,32)).type(ttype)
input_var

tensor([[[[-0.2973, -1.8580, -0.5975,  ...,  1.3263,  1.2916, -0.4290],
          [-1.0215,  0.0675, -0.9857,  ...,  1.3524,  0.6812,  1.5958],
          [ 0.0089,  0.3174, -0.6525,  ..., -1.4961,  0.6239, -0.9933],
          ...,
          [ 0.4883, -0.0547, -0.0974,  ...,  0.1020, -1.3684, -0.1422],
          [ 0.2412,  0.2797, -0.2057,  ...,  0.1744, -1.4662,  0.7479],
          [-1.8779,  1.6263, -1.6444,  ..., -0.3677,  0.8274,  0.8945]],

         [[ 1.6880,  1.1349, -0.0469,  ..., -0.2843,  0.0886, -0.8200],
          [-0.4533, -1.3211, -1.0611,  ..., -0.2144,  0.1663,  0.4091],
          [ 1.4487, -0.3952,  0.4396,  ...,  0.1303, -0.1160,  0.3651],
          ...,
          [-0.1381,  0.4277, -0.0694,  ...,  0.1845,  0.2456, -1.9692],
          [ 0.5448, -0.5923,  0.0232,  ..., -0.0760,  0.0342, -0.0051],
          [ 0.8865, -1.2380,  1.3131,  ...,  0.7514,  0.3448,  0.0068]],

         [[-1.6050, -0.4920,  0.0739,  ...,  0.7775,  0.2723, -0.1352],
          [-0.0201, -0.0703,  

In [36]:
model.eval()
quantizer.store_and_quantize(training=False, get_quant_params=False)

In [37]:
output = model(input_var)
quantizer.restore_real_value()
output.data

tensor([[-1.1488, -2.6569,  5.3355,  1.3420, -1.5785,  0.7806,  4.1762,  1.8382,
         -5.5024, -2.6558]], device='cuda:0')

In [29]:
quantizer.store_and_quantize(training=False, get_quant_params=True)

In [30]:
quantizer.deployment_model.type(ttype)
output2 = quantizer.deployment_model(input_var)
quantizer.restore_real_value()

output2.data

tensor([[ -30.6576,   21.5708,  117.1469,    6.1752, -111.7767,   37.9310,
          133.3673,   48.5023, -167.9324,  -54.4490]], device='cuda:0')

In [21]:
output.data/output2.data

tensor([[ 0.0227, -0.0039,  0.0264,  0.1494,  0.0201,  0.0206,  0.0225,  0.0312,
          0.0266,  0.0338]], device='cuda:0')

In [22]:
(output.data-output2.data).abs().sum()

tensor(711.0366, device='cuda:0')

In [23]:
net = nn.Sequential(model.module.model[0][0],model.module.model[0][1] )#,model.module.model[1])
output1 = net(input_var)
output1

tensor([[[[-1.2298e+00, -1.3090e+00, -1.2916e+00,  ..., -1.2027e+00,
           -1.1830e+00, -1.1214e+00],
          [-1.2267e+00, -1.2917e+00, -1.2363e+00,  ..., -1.1769e+00,
           -1.2008e+00, -9.8096e-01],
          [-1.1631e+00, -1.1992e+00, -1.2507e+00,  ..., -1.2457e+00,
           -1.2062e+00, -1.0203e+00],
          ...,
          [-1.1890e+00, -1.0999e+00, -1.1926e+00,  ..., -1.2107e+00,
           -1.2402e+00, -1.0884e+00],
          [-1.1694e+00, -1.0818e+00, -1.2422e+00,  ..., -1.3756e+00,
           -1.1469e+00, -1.1982e+00],
          [-1.1578e+00, -1.2190e+00, -1.2013e+00,  ..., -1.2658e+00,
           -1.2421e+00, -1.2590e+00]],

         [[ 1.3645e-02, -9.3586e-02, -8.3263e-02,  ..., -1.5779e-01,
            5.7411e-02, -6.6135e-02],
          [ 9.7727e-02,  1.1886e-02,  5.1301e-02,  ..., -1.0439e-01,
            6.8946e-02, -1.0653e-01],
          [-5.2667e-02,  1.0581e-01,  4.1952e-02,  ..., -2.0856e-01,
            2.7062e-01, -4.0397e-03],
          ...,
     

In [24]:
ptr_md = quantizer.deployment_model.model
net2= nn.Sequential( ptr_md[0])#,ptr_md[1])#,ptr_md[2],ptr_md[3])

In [26]:
output2 = net2(input_var)
output2

tensor([[[[-2.8973e+02, -3.0840e+02, -3.0429e+02,  ..., -2.8336e+02,
           -2.7872e+02, -2.6420e+02],
          [-2.8900e+02, -3.0432e+02, -2.9126e+02,  ..., -2.7727e+02,
           -2.8290e+02, -2.3111e+02],
          [-2.7401e+02, -2.8253e+02, -2.9466e+02,  ..., -2.9349e+02,
           -2.8418e+02, -2.4039e+02],
          ...,
          [-2.8012e+02, -2.5914e+02, -2.8097e+02,  ..., -2.8525e+02,
           -2.9218e+02, -2.5641e+02],
          [-2.7550e+02, -2.5487e+02, -2.9266e+02,  ..., -3.2409e+02,
           -2.7020e+02, -2.8228e+02],
          [-2.7278e+02, -2.8719e+02, -2.8301e+02,  ..., -2.9821e+02,
           -2.9263e+02, -2.9661e+02]],

         [[ 3.2147e+00, -2.2048e+01, -1.9617e+01,  ..., -3.7174e+01,
            1.3526e+01, -1.5581e+01],
          [ 2.3024e+01,  2.8003e+00,  1.2086e+01,  ..., -2.4594e+01,
            1.6244e+01, -2.5098e+01],
          [-1.2408e+01,  2.4930e+01,  9.8837e+00,  ..., -4.9137e+01,
            6.3758e+01, -9.5175e-01],
          ...,
     

In [47]:
output1[0][10]

tensor([[ 0.6668,  0.8130,  0.5452,  ...,  2.5753,  2.7961,  2.4325],
        [ 1.5222,  2.2391,  3.4420,  ...,  1.3467,  1.0082,  1.3457],
        [ 0.7315, -0.0835, -0.4004,  ...,  1.5123,  1.1432,  0.5446],
        ...,
        [ 1.2904,  1.1056,  0.8274,  ...,  0.3274,  1.6566,  0.8545],
        [ 0.9708,  0.5848,  1.1020,  ...,  0.4696,  1.5043,  1.8956],
        [ 1.3714,  1.2828,  0.9303,  ...,  1.4516,  0.2574,  1.8278]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [49]:
quantizer.deployment_model.model[0].weight.data[10]

tensor([[[-0.1085, -0.2053, -0.0891],
         [ 0.1201,  0.1860,  0.1317],
         [ 0.0116,  0.0116,  0.0271]],

        [[-0.2867, -0.4184, -0.2053],
         [ 0.1627,  0.2944,  0.1976],
         [ 0.0349,  0.0465, -0.0155]],

        [[-0.0542, -0.1356, -0.0310],
         [ 0.0930,  0.1278,  0.0659],
         [ 0.0271, -0.0155, -0.0271]]], device='cuda:0')

In [51]:
model.module.model[0][0].weight.data[10]

tensor([[[-0.3596, -0.6925, -0.2930],
         [ 0.3995,  0.6259,  0.4395],
         [ 0.0400,  0.0400,  0.0932]],

        [[-0.9588, -1.3983, -0.6925],
         [ 0.5460,  0.9855,  0.6659],
         [ 0.1199,  0.1598, -0.0533]],

        [[-0.1864, -0.4528, -0.1065],
         [ 0.3063,  0.4261,  0.2264],
         [ 0.0932, -0.0533, -0.0932]]], device='cuda:0')

In [44]:
(output1.data - output2.data).abs().sum()

tensor(9430.5840, device='cuda:0')

In [23]:
quantizer.deployment_model.model[0](input_var)

tensor([[[[-0.0045, -0.0045, -0.0045,  ..., -0.0045, -0.0045, -0.0045],
          [-0.0045, -0.0045, -0.0045,  ..., -0.0045, -0.0045, -0.0045],
          [-0.0045, -0.0045, -0.0045,  ..., -0.0045, -0.0045, -0.0045],
          ...,
          [-0.0045, -0.0045, -0.0045,  ..., -0.0045, -0.0045, -0.0045],
          [-0.0045, -0.0045, -0.0045,  ..., -0.0045, -0.0045, -0.0045],
          [-0.0045, -0.0045, -0.0045,  ..., -0.0045, -0.0045, -0.0045]],

         [[-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          ...,
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003]],

         [[-0.0171, -0.0171, -0.0171,  ..., -0.0171, -0.0171, -0.0171],
          [-0.0171, -0.0171, -

In [24]:
quantizer.deployment_model.model[0].bias.data = bias_tensor

NameError: name 'bias_tensor' is not defined

In [58]:
mu_tensor = model.module.model[0][1].running_mean.clone()
mu_tensor

tensor([ 9.2268e-03, -5.6052e-45, -1.3223e-01,  9.5181e-02,  5.6052e-45,
         2.7001e-02,  1.2026e-02,  5.6052e-45,  5.6052e-45,  5.6052e-45,
        -5.6052e-45, -1.5036e-01, -5.6052e-45, -5.6052e-45, -5.6052e-45,
         5.6052e-45, -5.6052e-45,  5.6052e-45, -1.6848e-01,  3.7778e-02,
         5.6052e-45,  5.6052e-45,  5.6052e-45, -5.6052e-45, -5.6052e-45,
         5.6052e-45,  8.0595e-03, -5.6052e-45, -1.1367e-01, -1.0080e-01,
         1.2784e-01,  4.8255e-03], device='cuda:0')

In [63]:
bias_tensor = mu_tensor.mul(-1)*gamma_over_sigma+beta_tensor
bias_tensor

tensor([ 1.1460e+00, -1.3018e-05,  9.6313e-01,  1.1933e+00, -3.9374e-04,
         1.1778e+00, -2.0396e-01, -2.6259e-04,  5.2023e-07, -2.0778e-06,
        -2.6015e-05,  1.1760e+00, -3.2594e-06, -1.3979e-05, -1.2620e-07,
         5.4683e-07, -1.1149e-05, -2.4389e-05,  8.3711e-01,  1.0547e+00,
        -6.0049e-05,  2.1387e-07,  9.6728e-07, -1.0179e-04, -2.4040e-05,
         4.7633e-06,  1.2126e+00, -1.0693e-05,  1.0251e+00,  6.2181e-01,
         8.9824e-01,  1.1250e+00], device='cuda:0')

In [61]:
beta_tensor = model.module.model[0][1].bias.data.clone()
beta_tensor

tensor([ 1.1506e+00, -1.3018e-05,  9.5048e-01,  1.2050e+00, -3.9374e-04,
         1.1813e+00, -2.0231e-01, -2.6259e-04,  5.2023e-07, -2.0778e-06,
        -2.6015e-05,  1.1557e+00, -3.2594e-06, -1.3979e-05, -1.2620e-07,
         5.4683e-07, -1.1149e-05, -2.4389e-05,  8.1169e-01,  1.0597e+00,
        -6.0049e-05,  2.1387e-07,  9.6728e-07, -1.0179e-04, -2.4040e-05,
         4.7633e-06,  1.2149e+00, -1.0693e-05,  1.0384e+00,  6.1218e-01,
         9.1894e-01,  1.1263e+00], device='cuda:0')

In [52]:
gamma_tensor = model.module.model[0][1].weight.data.clone()

In [53]:
var_tensor = model.module.model[0][1].running_var.clone()

In [54]:
weight = model.module.model[0][0].weight.data.clone()
weight

tensor([[[[-0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000]],

         [[-0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000]],

         [[-0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000]],

         [[-0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000]],

         [[ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
     

In [55]:
v0,v1,v2,v3 = weight.size()

In [56]:
eps = model.module.model[0][1].eps

In [57]:
sigma_tensor = var_tensor.add(eps).sqrt()

In [11]:
LAST = len(quantizer_load.param_to_quantize) -1

In [114]:
# x = torch.Tensor(1,3,int(input_size),int(input_size))
#compute input dimensions
def print_size(model, input, output): 
    global si, so
    si = input[0].size()
    so = output[0].size()

activation_vector_i = []
activation_vector_o = []

for i,module in enumerate(model.modules()):
    if type(module) is models.linear_quantized_modules.Conv2d_SAME:
        print('Conv2d Same =: ',i)
        hook = module.register_forward_hook(print_size)
        model(x)
        hook.remove() 
        activation_vector_i.append(si)
        activation_vector_o.append(so)
        
    elif type(module) is nn.Linear:
        print('Linear =: ',i)
        hook = module.register_forward_hook(print_size)
        model(x)
        hook.remove() 
        activation_vector_i.append(si)
        activation_vector_o.append(so)

    elif type(module) is nn.Conv2d:
        print('Normal Conv =: ',i)

def compute_activation_footprint(param_list):
    i_params_mem = []
    i_bits_mem = []
    o_params_mem = []
    o_bits_mem = []

    for i,item in enumerate(param_list):
        o_size = activation_vector_o[i]
        i_size = activation_vector_i[i]
        o_params = 1
        for v in range(len(o_size)):
            o_params *= o_size[v]
        i_params = 1
        for v in range(len(i_size)):
            i_params *= i_size[v] 
        
        if i == 0:
            i_bits = 8
            o_bits = param_list[i]['act_o_bits']
        elif i == LAST:
            i_bits = param_list[i-1]['act_o_bits']
            o_bits = 8
        else:
            i_bits = param_list[i-1]['act_o_bits']
            o_bits = param_list[i]['act_o_bits']
        
        i_mem = i_bits * i_params / 32
        o_mem = o_bits * o_params / 32
            
        i_params_mem.append(i_params)
        i_bits_mem.append(i_bits)
        o_params_mem.append(o_params)
        o_bits_mem.append(o_bits)
        
        #print('Input = ', i_params,'(bits = ', i_bits, ') | Output Params = ', o_params,'(bits = ', o_bits,')')            

    return i_params_mem, i_bits_mem, o_params_mem, o_bits_mem

def cut_activation_footprint(param_list, act_mem):
    MIN_ACT_BITS = 2
    i_params_mem, i_bits_mem, o_params_mem, o_bits_mem = compute_activation_footprint(param_list)
    errQuant = False
    for i in range(len(i_params_mem)):
        print('*********** Layer ',i, 'Start **************')
        tot_layer_mem = ((i_params_mem[i] * i_bits_mem[i]) + (o_params_mem[i] * o_bits_mem[i]) )/32
        n_iter = 1
        while tot_layer_mem > act_mem:
            
            print('Iteration ',n_iter, 'with bits: ',i_bits_mem[i],o_bits_mem[i] )
            
            i_mem = i_params_mem[i] * i_bits_mem[i]
            o_mem = o_params_mem[i] * o_bits_mem[i]

            if i == 0: # first layer
                if o_bits_mem[i] > MIN_ACT_BITS :
                    param_list[i]['act_o_bits'] = int(o_bits_mem[i] / 2)
                else:
                    errQuant = True
                    print('No way to find a solution because of the first layer!')
                    break
            elif i == LAST: # last layer
                if i_bits_mem[i] > MIN_ACT_BITS :
                    param_list[i-1]['act_o_bits'] = int(i_bits_mem[i] / 2)
                else:
                    errQuant = True
                    print('No way to find a solution because of the last layer!')
                    break                
            else:
                if i_bits_mem[i] == MIN_ACT_BITS and o_bits_mem[i] == MIN_ACT_BITS :
                    errQuant = True
                    print('No way to find a solution: layer ',i,'with i_bits,o_bits =',MIN_ACT_BITS)
                    break

                elif i_bits_mem[i] > o_bits_mem[i] and i_bits_mem[i] > MIN_ACT_BITS:
                    param_list[i-1]['act_o_bits'] = int(i_bits_mem[i] / 2)     

                elif i_bits_mem[i] < o_bits_mem[i] and o_bits_mem[i] > MIN_ACT_BITS:
                    param_list[i]['act_o_bits'] = int(o_bits_mem[i] / 2)  

                elif i_bits_mem[i] == o_bits_mem[i]:
                    if i_mem > o_mem and i_bits_mem[i] > MIN_ACT_BITS :
                        param_list[i-1]['act_o_bits'] = int(i_bits_mem[i] / 2)
                    elif o_bits_mem[i] > MIN_ACT_BITS:
                        param_list[i]['act_o_bits'] = int(o_bits_mem[i] / 2)
                    else:
                        print('Corner case!')

                elif o_bits_mem[i] > MIN_ACT_BITS :
                    param_list[i]['act_o_bits'] = int(o_bits_mem[i] / 2)

                elif i_bits_mem[i] > MIN_ACT_BITS :
                    param_list[i-1]['act_o_bits'] = int(i_bits_mem[i] / 2)

                else: 
                    errQuant = True
                    print('No way to find a solution!')
                    break                

            i_params_mem, i_bits_mem, o_params_mem, o_bits_mem = compute_activation_footprint(param_list)
            tot_layer_mem = ((i_params_mem[i] * i_bits_mem[i]) + (o_params_mem[i] * o_bits_mem[i]) )/32
            n_iter +=1



        else:
            print('Layer ',i,' is OK')
            
    i_params_mem, i_bits_mem, o_params_mem, o_bits_mem = compute_activation_footprint(param_list)
    print('Input: ',i_bits_mem,'\n  Output:', o_bits_mem)
    if errQuant is True:
        print('No way for quantization')
        
        
    
            

            
ONLY_READ_MEM = 64*1024
param_list = copy.deepcopy(quantizer_load.param_to_quantize)
tt = cut_activation_footprint(param_list, ONLY_READ_MEM)
print(tt)

Conv2d Same =:  3
Conv2d Same =:  7
Conv2d Same =:  11
Conv2d Same =:  15
Conv2d Same =:  19
Conv2d Same =:  23
Conv2d Same =:  27
Conv2d Same =:  31
Conv2d Same =:  35
Conv2d Same =:  39
Conv2d Same =:  43
Conv2d Same =:  47
Conv2d Same =:  51
Conv2d Same =:  55
Conv2d Same =:  59
Conv2d Same =:  63
Conv2d Same =:  67
Conv2d Same =:  71
Conv2d Same =:  75
Conv2d Same =:  79
Conv2d Same =:  83
Conv2d Same =:  87
Conv2d Same =:  91
Conv2d Same =:  95
Conv2d Same =:  99
Conv2d Same =:  103
Conv2d Same =:  107
Linear =:  111
*********** Layer  0 Start **************
Layer  0  is OK
*********** Layer  1 Start **************
Iteration  1 with bits:  8 8
Layer  1  is OK
*********** Layer  2 Start **************
Iteration  1 with bits:  4 8
Layer  2  is OK
*********** Layer  3 Start **************
Layer  3  is OK
*********** Layer  4 Start **************
Layer  4  is OK
*********** Layer  5 Start **************
Iteration  1 with bits:  8 8
Layer  5  is OK
*********** Layer  6 Start **********

In [43]:

def compute_params_footprint(param_list):
    LAST = len(param_list)
    weight_footprint = [0 for x in range(LAST)]
    weight_bits = [0 for x in range(LAST)]
    for i,item in enumerate(param_list):
        #number of bits
        w_bits = item['w_bits']
                
        # size of convolutional parameters
        len_w = len(item['weight'].size())
        size_param = 1
        for v in range(len_w):
            size_param *= item['weight'].size(v)
        w_footprint = (size_param * w_bits) / 8 #measure in bytes
        weight_footprint[i] = w_footprint
        weight_bits[i] = w_bits
        
        #print('------------------- This is the layer ', i, '---------------')
        #print('Weight Bits: ',w_bits)
        #print('Number of parameters: ',size_param)
        #print('Parameters footprint: ',w_footprint)
        #if i is LAST-1:
        #    print('no')
        #else:
        #    print('Activation Bits = ',item['act_o_bits'])
    
    return weight_footprint,sum(weight_footprint), weight_bits


def compute_bias_footprint(param_list):
    BIAS_PACT_BITS = 32
    BIAS_PACTCH_BITS = 8
    BIAS_M0_BITS = 32
    BIAS_N0_BITS = 8

    LAST = len(param_list)
    weight_footprint = [0 for x in range(LAST)]
    for i,item in enumerate(param_list):

        #number of bits
        w_bits = item['w_bits']
        n_out_ch = item['weight'].size(0)
        
        # size of extra parametes
        quant_type = item['quant_type']
        fold_type = item['fold_type']
        if quant_type is None:
            quant_type = 'PACT'
            fold_type = 'folding_weights'
        elif w_bits < 8:
            quant_type = 'PACT_CHANNEL'
            fold_type = 'fixed_batch'
        item['quant_type'] = quant_type
        item['fold_type'] = fold_type
        
        # compute number of bias
        if quant_type == 'PACT':
            bias_size = int(n_out_ch * BIAS_PACT_BITS / 8 )
        elif quant_type == 'PACT_CHANNEL':
            bias_size  = int(n_out_ch * BIAS_M0_BITS / 8 ) # M0 
            bias_size += int(n_out_ch * BIAS_N0_BITS / 8 ) # N0 
            bias_size += int(n_out_ch * BIAS_PACTCH_BITS / 8 ) # B 
            bias_size += int(n_out_ch * BIAS_PACTCH_BITS / 8 ) # ZW 

        else:
            print('ERROR!!!')
        
        weight_footprint[i] = bias_size
        
    return weight_footprint

def compute_footprint(param_list):
    weight_footprint, _, _ = compute_params_footprint(param_list)
    bias_footprint = compute_bias_footprint(param_list)
    ratio = np.array(bias_footprint) / np.array(weight_footprint)
    weight_footprint = sum(weight_footprint)
    bias_footprint = sum(bias_footprint)
    print('Total Weights: ',int(weight_footprint/1024),'kbytes | bias =' ,int(bias_footprint/1024),'kbytes ('\
          ,100*bias_footprint/weight_footprint ,'%)')
    #print('max:', ratio * 100,'%')
    return bias_footprint+weight_footprint
        
        
def compute_next_cut(param_list):
    MAX = 0.05

    f,_,w_b = compute_params_footprint(param_list)
    weight = np.array(f)
    total = weight.sum()
    perc = weight / total
    arr = np.sort(perc)[::-1]
    ind = np.argsort(perc)[::-1]
    
    # remove w_bit = 1 from search
    rm = []
    for i,item in enumerate(w_b):
        if item == MIN_BIT:
            for i_x,x in enumerate(ind):
                if x == i:
                    rm.append(i_x)
                    continue
    print(rm)
    arr = np.delete(arr, rm)
    ind = np.delete(ind, rm)
    
    if len(arr) == 0:
        return -1
        
    #print(arr, ind)
    LAST = len(arr)

    cut_we = arr[0]
    cut_i = ind[0]
    print('most footprint on layer: ',cut_i, 'equal to', cut_we, 'ratio' )
    thr = cut_we - MAX
    for i in range(1, LAST):
        if arr[i] > thr and ind[i] < cut_i:
            cut_i = ind[i]
            print(ind[i],arr[i])
    return(cut_i)            

    
def memory_driven_quantization(param_list, read_only_mem):
    _, s, _ = compute_params_footprint(param_list)
    print('Total Footprint: ', s )
    while (s>read_only_mem):
        print('***************************************************************')
        c = compute_next_cut(param_list)
        if c == -1:
            return -1
        nb = int(param_list[c]['w_bits'] / 2)
        print('Layer to cut: ', c, 'to ', nb  ,'bits')
        param_list[c]['w_bits'] = nb
        #_, s, _ = compute_params_footprint(param_list)
        s = compute_footprint(param_list)
    return(param_list)


            


ONLY_READ_MEM = 512*1024
param_list = copy.deepcopy(quantizer_load.param_to_quantize)
param_list = memory_driven_quantization(param_list, ONLY_READ_MEM)
if param_list == -1:
    print('No way')
else:
    _, s, w_bits = compute_params_footprint(param_list)
    print('-----------  Total Footprint: ', s )
    print('Final Quantization: ', w_bits )


Total Footprint:  2568144.0
***************************************************************
[]
most footprint on layer:  27 equal to 0.299048651477 ratio
Layer to cut:  27 to  4 bits
Total Weights:  2132 kbytes | bias = 38 kbytes ( 1.823689280560256 %)
***************************************************************
[]
most footprint on layer:  26 equal to 0.270048128695 ratio
Layer to cut:  26 to  4 bits
Total Weights:  1844 kbytes | bias = 41 kbytes ( 2.230324279919036 %)
***************************************************************
[]
most footprint on layer:  27 equal to 0.203257196575 ratio
26 0.15610152697
24 0.15610152697
Layer to cut:  24 to  4 bits
Total Weights:  1700 kbytes | bias = 43 kbytes ( 2.551418781749203 %)
***************************************************************
[]
most footprint on layer:  27 equal to 0.220464629206 ratio
Layer to cut:  27 to  2 bits
Total Weights:  1513 kbytes | bias = 43 kbytes ( 2.867511175808633 %)
**************************************

In [58]:
sigma_tensor

tensor([0.0032, 0.0032, 0.0032, 0.0032, 5.1141, 0.1086, 4.4318, 3.4280, 3.2614,
        0.0032, 1.4061, 0.0032, 1.3680, 5.7485, 0.0032, 0.0032, 0.0032, 0.0074,
        1.0969, 3.7479, 1.9707, 4.7107, 1.5083, 2.8547, 4.3182, 0.0032, 2.3813,
        0.0032, 0.0868, 1.5090, 4.2023, 0.0032], device='cuda:0')

In [59]:
gamma_over_sigma = gamma_tensor / sigma_tensor
gamma_over_sigma

tensor([ 0.0080,  0.6109,  0.4207,  0.6073,  0.1026,  0.0430,  0.1581,  0.2290,
         0.1794,  0.2899,  0.2980,  1.3301,  0.3118,  0.1344,  0.2803, -0.8490,
         0.6654, -0.8486, -0.3533,  0.2579,  0.1002,  0.0991,  0.1978,  0.2172,
         0.1020,  0.0547,  0.1285,  0.7153, -0.0120,  0.0705,  0.2167, -0.1538],
       device='cuda:0')

In [60]:
gamma_over_sigma_tensor = gamma_over_sigma.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(v0,v1,v2,v3 )
gamma_over_sigma_tensor

tensor([[[[ 0.0080,  0.0080,  0.0080],
          [ 0.0080,  0.0080,  0.0080],
          [ 0.0080,  0.0080,  0.0080]],

         [[ 0.0080,  0.0080,  0.0080],
          [ 0.0080,  0.0080,  0.0080],
          [ 0.0080,  0.0080,  0.0080]],

         [[ 0.0080,  0.0080,  0.0080],
          [ 0.0080,  0.0080,  0.0080],
          [ 0.0080,  0.0080,  0.0080]]],


        [[[ 0.6109,  0.6109,  0.6109],
          [ 0.6109,  0.6109,  0.6109],
          [ 0.6109,  0.6109,  0.6109]],

         [[ 0.6109,  0.6109,  0.6109],
          [ 0.6109,  0.6109,  0.6109],
          [ 0.6109,  0.6109,  0.6109]],

         [[ 0.6109,  0.6109,  0.6109],
          [ 0.6109,  0.6109,  0.6109],
          [ 0.6109,  0.6109,  0.6109]]],


        [[[ 0.4207,  0.4207,  0.4207],
          [ 0.4207,  0.4207,  0.4207],
          [ 0.4207,  0.4207,  0.4207]],

         [[ 0.4207,  0.4207,  0.4207],
          [ 0.4207,  0.4207,  0.4207],
          [ 0.4207,  0.4207,  0.4207]],

         [[ 0.4207,  0.4207,  0.4207],
     

In [61]:
weight_tensor = weight * gamma_over_sigma_tensor

In [75]:
weight_tensor[10].div(d255).round().mul(d255)

tensor([[[-0.1085, -0.2053, -0.0891],
         [ 0.1201,  0.1860,  0.1317],
         [ 0.0116,  0.0116,  0.0271]],

        [[-0.2867, -0.4184, -0.2053],
         [ 0.1627,  0.2944,  0.1976],
         [ 0.0349,  0.0465, -0.0155]],

        [[-0.0542, -0.1356, -0.0310],
         [ 0.0930,  0.1278,  0.0659],
         [ 0.0271, -0.0155, -0.0271]]], device='cuda:0')

In [74]:
delta = weight_tensor.data.max()-weight_tensor.data.min()
d255 = delta / 255
d255

tensor(0.0039, device='cuda:0')

In [67]:
quantizer.deployment_model.model[0].weight.data[10]

tensor([[[-0.1085, -0.2053, -0.0891],
         [ 0.1201,  0.1860,  0.1317],
         [ 0.0116,  0.0116,  0.0271]],

        [[-0.2867, -0.4184, -0.2053],
         [ 0.1627,  0.2944,  0.1976],
         [ 0.0349,  0.0465, -0.0155]],

        [[-0.0542, -0.1356, -0.0310],
         [ 0.0930,  0.1278,  0.0659],
         [ 0.0271, -0.0155, -0.0271]]], device='cuda:0')

In [51]:
loss = criterion(output, target_var)

In [52]:
loss.data

tensor(5.6001, device='cuda:0')

In [53]:
optimizer.zero_grad()

In [54]:
loss.backward()

In [55]:
quantizer.restore_real_value()            
quantizer.backprop_quant_gradients()  

In [56]:
optimizer.step()

In [36]:
quantizer.deployment_model.model[1].weight.data

AttributeError: 'ReLU6' object has no attribute 'weight'

In [40]:
model.model[0][1].running_var


 2.9390e-03
 6.2338e-04
 5.6052e-45
 1.7143e-01
 1.7013e-02
 4.5158e-02
 2.7970e-03
 7.9009e-02
 1.0388e-01
 5.0715e-02
 4.5062e-02
 3.5231e-01
 8.3555e-03
 9.0936e-05
 2.9523e-02
 8.0683e-01
 4.9630e-03
 5.5362e-01
 5.4003e-03
 9.3962e-03
 5.6052e-45
 7.3168e-04
 1.3904e-03
 8.2610e-03
 9.0941e-04
 1.1302e-02
 5.6052e-45
 1.0920e-02
 7.0269e-02
 1.7943e-03
 8.2472e-03
 4.1882e-02
[torch.cuda.FloatTensor of size 32 (GPU 2)]

In [21]:
output2 = quantizer.deployment_model.model(input_var)
output2

Variable containing:
( 0  , 0  ,.,.) = 
  0.0022

( 0  , 1  ,.,.) = 
  0.2660

( 0  , 2  ,.,.) = 
  0.0000
      ... 

( 0  ,1021,.,.) = 
  0.0112

( 0  ,1022,.,.) = 
  0.0024

( 0  ,1023,.,.) = 
  0.0000
        ⋮  

( 1  , 0  ,.,.) = 
  0.0022

( 1  , 1  ,.,.) = 
  0.2753

( 1  , 2  ,.,.) = 
  0.0000
      ... 

( 1  ,1021,.,.) = 
  0.0117

( 1  ,1022,.,.) = 
  0.0023

( 1  ,1023,.,.) = 
  0.0000
        ⋮  

( 2  , 0  ,.,.) = 
  0.0022

( 2  , 1  ,.,.) = 
  0.2900

( 2  , 2  ,.,.) = 
  0.0000
      ... 

( 2  ,1021,.,.) = 
  0.0102

( 2  ,1022,.,.) = 
  0.0023

( 2  ,1023,.,.) = 
  0.0000
 ...      
        ⋮  

( 61 , 0  ,.,.) = 
  0.0021

( 61 , 1  ,.,.) = 
  0.2783

( 61 , 2  ,.,.) = 
  0.0000
      ... 

( 61 ,1021,.,.) = 
  0.0094

( 61 ,1022,.,.) = 
  0.0023

( 61 ,1023,.,.) = 
  0.0000
        ⋮  

( 62 , 0  ,.,.) = 
  0.0022

( 62 , 1  ,.,.) = 
  0.2816

( 62 , 2  ,.,.) = 
  0.0000
      ... 

( 62 ,1021,.,.) = 
  0.0087

( 62 ,1022,.,.) = 
  0.0024

( 62 ,1023,.,.) = 
  0.0

In [23]:
quantizer.deployment_model.model[0].weight.data


(0 ,0 ,.,.) = 
  0.0000  0.0045  0.0045
  0.0045  0.0045  0.0045
  0.0000  0.0045  0.0045

(0 ,1 ,.,.) = 
 -0.0045 -0.0045 -0.0000
 -0.0045 -0.0000 -0.0045
 -0.0000 -0.0000 -0.0000

(0 ,2 ,.,.) = 
  0.0000  0.0045  0.0000
  0.0000  0.0000  0.0045
  0.0000  0.0045  0.0045

(1 ,0 ,.,.) = 
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000

(1 ,1 ,.,.) = 
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000

(1 ,2 ,.,.) = 
  0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000

(2 ,0 ,.,.) = 
  0.0180  0.0271 -0.0225
 -0.0135 -0.0406 -0.0406
 -0.0361 -0.0406 -0.0000

(2 ,1 ,.,.) = 
  0.0406  0.0180 -0.0271
 -0.0406 -0.0857 -0.0722
 -0.0496 -0.0496 -0.0000

(2 ,2 ,.,.) = 
  0.0406  0.0225 -0.0271
 -0.0361 -0.0406 -0.0451
 -0.0406 -0.0271  0.0045

(3 ,0 ,.,.) = 
 -0.0000 -0.0135 -0.0000
 -0.0000 -0.0045 -0.0045
 -0.0045  0.0045 -0.0045

(3 ,1 ,.,.) = 
 -0.0135 -0.0045 -0.0000
 -0.0135 -0.0000 -0.0045
  0.0000  0.0000  0.0045

In [24]:
model.model[0][0].weight.data


(0 ,0 ,.,.) = 
  0.0000  0.0838  0.0838
  0.0838  0.0838  0.0838
  0.0000  0.0838  0.0838

(0 ,1 ,.,.) = 
 -0.0838 -0.0838 -0.0000
 -0.0838 -0.0000 -0.0838
 -0.0000 -0.0000 -0.0000

(0 ,2 ,.,.) = 
  0.0000  0.0838  0.0000
  0.0000  0.0000  0.0838
  0.0000  0.0838  0.0838

(1 ,0 ,.,.) = 
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000

(1 ,1 ,.,.) = 
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000

(1 ,2 ,.,.) = 
  0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000

(2 ,0 ,.,.) = 
  0.1329  0.1994 -0.1662
 -0.0997 -0.2991 -0.2991
 -0.2659 -0.2991 -0.0000

(2 ,1 ,.,.) = 
  0.2991  0.1329 -0.1994
 -0.2991 -0.6314 -0.5317
 -0.3656 -0.3656 -0.0000

(2 ,2 ,.,.) = 
  0.2991  0.1662 -0.1994
 -0.2659 -0.2991 -0.3323
 -0.2991 -0.1994  0.0332

(3 ,0 ,.,.) = 
 -0.0000 -0.0540 -0.0000
 -0.0000 -0.0180 -0.0180
 -0.0180  0.0180 -0.0180

(3 ,1 ,.,.) = 
 -0.0540 -0.0180 -0.0000
 -0.0540 -0.0000 -0.0180
  0.0000  0.0000  0.0180

In [31]:
cc.zero_()

AttributeError: 'torch.FloatTensor' object has no attribute 'zeros'

In [30]:
torch.clamp()

NameError: name 'clip' is not defined

In [25]:
output = nn.Sequential(model.model[0][0],model.model[0][1]) (input_var)
output

Variable containing:
( 0 , 0 ,.,.) = 
 -2.0669e-01 -2.0669e-01 -2.0669e-01  ...  -2.0669e-01 -2.0669e-01 -2.0669e-01
 -2.0669e-01 -2.0669e-01 -2.0669e-01  ...  -2.0669e-01 -2.0669e-01 -2.0669e-01
 -2.0669e-01 -2.0669e-01 -2.0669e-01  ...  -2.0669e-01 -2.0669e-01 -2.0669e-01
                 ...                   ⋱                   ...                
 -2.0669e-01 -2.0669e-01 -2.0669e-01  ...  -2.0669e-01 -2.0669e-01 -2.0669e-01
 -2.0669e-01 -2.0669e-01 -2.0669e-01  ...  -2.0669e-01 -2.0669e-01 -2.0669e-01
 -2.0669e-01 -2.0669e-01 -2.0669e-01  ...  -2.0669e-01 -2.0669e-01 -2.0669e-01

( 0 , 1 ,.,.) = 
  3.1326e-02  3.1326e-02  3.1326e-02  ...   3.1326e-02  3.1326e-02  3.1326e-02
  3.1326e-02  3.1326e-02  3.1326e-02  ...   3.1326e-02  3.1326e-02  3.1326e-02
  3.1326e-02  3.1326e-02  3.1326e-02  ...   3.1326e-02  3.1326e-02  3.1326e-02
                 ...                   ⋱                   ...                
  3.1326e-02  3.1326e-02  3.1326e-02  ...   3.1326e-02  3.1326e-02  3.1326e

In [25]:
cc

<function FloatTensor.clone>

In [19]:
#multi gpus
if gpus and len(gpus) > 1:
    model_dist = torch.nn.DataParallel(model, gpus)

In [20]:
output = model_dist(input_var)

In [21]:
quantizer.store_and_quantize()

In [25]:
quantizer.deployment_model(input_var)

Variable containing:
( 0 , 0 ,.,.) = 
  1.3363e+00  1.3145e+00  1.3089e+00  ...   1.3162e+00  1.3258e+00  1.3259e+00
  1.3862e+00  1.3369e+00  1.3300e+00  ...   1.3595e+00  1.3505e+00  1.3573e+00
  1.3731e+00  1.3275e+00  1.3182e+00  ...   1.3518e+00  1.3675e+00  1.3635e+00
                 ...                   ⋱                   ...                
  9.3571e-01  9.7869e-01  8.4257e-01  ...   8.7363e-01  8.7672e-01  8.8933e-01
  9.8799e-01  8.2247e-01  6.9976e-01  ...   8.5056e-01  8.6704e-01  8.7351e-01
  8.6465e-01  7.6496e-01  7.3176e-01  ...   8.5165e-01  8.8781e-01  8.9473e-01

( 0 , 1 ,.,.) = 
  2.3497e+00  2.5492e+00  2.5428e+00  ...   2.5717e+00  2.5678e+00  2.5981e+00
  2.3835e+00  2.7356e+00  2.7238e+00  ...   2.7793e+00  2.7772e+00  2.7869e+00
  2.3540e+00  2.7192e+00  2.6982e+00  ...   2.7431e+00  2.7900e+00  2.8017e+00
                 ...                   ⋱                   ...                
  1.8327e+00  2.1647e+00  2.1655e+00  ...   1.4009e+00  1.4049e+00  1.4236e

In [22]:
model_quant = torch.nn.DataParallel(quantizer.deployment_model, gpus)

In [23]:
output2 = model_quant(input_var)

In [24]:
output2

Variable containing:
 1.1552e+00  9.5017e-01  8.7872e-01  ...  -1.8631e-01  1.9957e+00  9.9282e-01
-3.9337e+00 -4.2530e+00 -1.2152e-01  ...  -2.8191e+00 -7.4017e-01  1.6144e+00
-5.5881e+00  3.0396e+00 -5.6240e-01  ...  -7.4701e-01 -1.7197e+00  4.4811e+00
                ...                   ⋱                   ...                
 1.9808e+00  2.0554e+00  1.9562e+00  ...   6.3680e-01  1.1966e+00 -4.0874e-01
 4.5948e-01 -1.6623e+00 -3.7264e-01  ...  -1.6432e+00  9.9374e-01  2.9577e+00
-9.2918e-01 -3.6385e+00  2.4017e+00  ...  -1.0772e+00 -1.9507e-01 -2.8710e-02
[torch.cuda.FloatTensor of size 128x1000 (GPU 2)]

In [25]:
output

Variable containing:
 1.9494e+00 -3.9579e-01  3.2307e-01  ...   4.3272e-01  2.4295e+00  3.2377e-01
-5.2903e+00 -5.2021e+00  9.0071e-03  ...  -3.1000e+00 -1.9988e+00  2.0122e+00
-6.1509e+00  2.0534e+00 -1.4551e+00  ...  -3.8637e-01 -2.2865e+00  4.8162e+00
                ...                   ⋱                   ...                
 1.6031e+00 -1.6777e+00 -1.1769e+00  ...   1.7391e+00 -1.1841e-01 -1.9975e+00
 2.4302e+00 -2.1855e+00  4.6637e-01  ...  -2.0303e+00  1.1533e+00  5.9466e-01
-6.8305e-01 -4.2784e+00  1.8464e+00  ...  -1.1086e+00  1.0207e-01 -1.4453e+00
[torch.cuda.FloatTensor of size 128x1000 (GPU 2)]

##### Evaluation ########

In [14]:
def forward(data_loader, model, criterion, epoch=0, training=True, optimizer=None, quantizer=None, get_quant_params=False):

#    if args.gpus and len(args.gpus) > 1:
#        model = torch.nn.DataParallel(model, args.gpus)

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end = time.time()


    for i, (inputs, target) in enumerate(data_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        if gpus is not None:
            target = target.cuda(async=True)
        input_var = Variable(inputs.type(ttype), volatile=not training)
        target_var = Variable(target)


        # quantization before computing output
        if quantizer is not None:
            quantizer.store_and_quantize(training=training, get_quant_params=False)

        # compute output
        output = model(input_var)

        loss = criterion(output, target_var)
        if type(output) is list:
            output = output[0]

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        top5.update(prec5[0], inputs.size(0))

        if training:
            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()

            # restore real value parameters before update
            if quantizer is not None:
                quantizer.restore_real_value()            
                quantizer.backprop_quant_gradients()    

            optimizer.step()
            
        elif quantizer is not None:
            quantizer.restore_real_value()


        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            logging.info('{phase} - Epoch: [{0}][{1}/{2}]\t'
                         'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                         'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                         'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                         'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                         'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                             epoch, i, len(data_loader),
                             phase='TRAINING' if training else 'EVALUATING',
                             batch_time=batch_time,
                             data_time=data_time, loss=losses, top1=top1, top5=top5))

    return losses.avg, top1.avg, top5.avg


def train(data_loader, model, criterion, epoch, optimizer, quantizer):
    # switch to train mode
    model.train()
    return forward(data_loader, model, criterion, epoch,
                   training=True, optimizer=optimizer, quantizer=quantizer, get_quant_params=False)


def validate(data_loader, model, criterion, epoch, quantizer, get_quant_params=False):
    # switch to evaluate mode
    model.eval()
    return forward(data_loader, model, criterion, epoch,
                   training=False, optimizer=None, quantizer=quantizer, get_quant_params=get_quant_params)

In [ ]:
val_loss, val_prec1, val_prec5 = validate(val_loader, model, criterion, 0, quantizer)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
EVALUATING - Epoch: [0][0/5000]	Time 1.692 (1.692)	Data 1.321 (1.321)	Loss 1.8149 (1.8149)	Prec@1 60.000 (60.000)	Prec@5 60.000 (60.000)
EVALUATING - Epoch: [0][10/5000]	Time 0.061 (0.206)	Data 0.000 (0.12

EVALUATING - Epoch: [0][540/5000]	Time 0.057 (0.058)	Data 0.000 (0.004)	Loss 2.0687 (2.8226)	Prec@1 40.000 (40.129)	Prec@5 70.000 (64.787)
EVALUATING - Epoch: [0][550/5000]	Time 0.054 (0.058)	Data 0.000 (0.004)	Loss 3.6695 (2.8129)	Prec@1 20.000 (40.109)	Prec@5 60.000 (64.991)
EVALUATING - Epoch: [0][560/5000]	Time 0.048 (0.058)	Data 0.000 (0.004)	Loss 3.2118 (2.8152)	Prec@1 50.000 (40.000)	Prec@5 60.000 (65.045)
EVALUATING - Epoch: [0][570/5000]	Time 0.052 (0.058)	Data 0.000 (0.004)	Loss 4.9135 (2.8378)	Prec@1 10.000 (39.667)	Prec@5 20.000 (64.676)
EVALUATING - Epoch: [0][580/5000]	Time 0.055 (0.058)	Data 0.000 (0.004)	Loss 2.3143 (2.8488)	Prec@1 40.000 (39.449)	Prec@5 80.000 (64.509)
EVALUATING - Epoch: [0][590/5000]	Time 0.050 (0.058)	Data 0.000 (0.003)	Loss 3.0146 (2.8411)	Prec@1 30.000 (39.662)	Prec@5 70.000 (64.619)
EVALUATING - Epoch: [0][600/5000]	Time 0.051 (0.058)	Data 0.000 (0.003)	Loss 2.1911 (2.8642)	Prec@1 60.000 (39.201)	Prec@5 60.000 (64.193)
EVALUATING - Epoch: [0][610

EVALUATING - Epoch: [0][1130/5000]	Time 0.056 (0.062)	Data 0.000 (0.006)	Loss 4.8854 (2.8243)	Prec@1 10.000 (37.409)	Prec@5 30.000 (64.951)
EVALUATING - Epoch: [0][1140/5000]	Time 0.057 (0.061)	Data 0.000 (0.006)	Loss 3.6886 (2.8326)	Prec@1 30.000 (37.309)	Prec@5 60.000 (64.812)
EVALUATING - Epoch: [0][1150/5000]	Time 0.056 (0.061)	Data 0.000 (0.006)	Loss 1.3710 (2.8288)	Prec@1 50.000 (37.341)	Prec@5 90.000 (64.857)
EVALUATING - Epoch: [0][1160/5000]	Time 0.061 (0.061)	Data 0.000 (0.006)	Loss 2.5221 (2.8218)	Prec@1 50.000 (37.347)	Prec@5 60.000 (64.961)
EVALUATING - Epoch: [0][1170/5000]	Time 0.059 (0.061)	Data 0.000 (0.006)	Loss 2.4675 (2.8273)	Prec@1 50.000 (37.199)	Prec@5 70.000 (64.842)
EVALUATING - Epoch: [0][1180/5000]	Time 0.058 (0.061)	Data 0.000 (0.006)	Loss 3.1728 (2.8266)	Prec@1 30.000 (37.172)	Prec@5 60.000 (64.936)
EVALUATING - Epoch: [0][1190/5000]	Time 0.064 (0.061)	Data 0.000 (0.006)	Loss 2.8026 (2.8265)	Prec@1 20.000 (37.145)	Prec@5 60.000 (64.962)
EVALUATING - Epoch: 

In [28]:
model.model[0][0].weight.data.mean()

0.09921024739742279

In [16]:
# Run validation
val_loss, val_prec1, val_prec5 = validate(val_loader, model, criterion, 0, quantizer, gpus)
print(val_loss, val_prec1, val_prec5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
EVALUATING - Epoch: [0][0/1563]	Time 12.467 (12.467)	Data 0.731 (0.731)	Loss 0.7861 (0.7861)	Prec@1 87.500 (87.500)	Prec@5 87.500 (87.500)
EVALUATING - Epoch: [0][10/1563]	Time 

  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 104, in get
    if timeout < 0 or not self._poll(timeout):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 104, in get
    if timeout < 0 or not self._poll(timeout):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 104, in get
    if timeout < 0 or not self._poll(timeout):
  File "/usr/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 104, in get
    if timeout

KeyboardInterrupt: 

In [24]:
model.module.model[0][1].running_var

tensor([13.2059,  5.9497,  1.0535, 10.2415, 28.1148, 12.4720,  3.4225,  2.6865,
        11.5736,  1.9509,  5.7577,  3.0689,  5.4406,  1.8761, 16.6637,  0.1006,
         7.2505, 17.0050,  3.0287, 10.2209,  2.5811,  2.2394, 10.0781, 19.4257,
         0.2328,  6.2219,  0.2076,  7.0416,  3.6740, 17.6075,  9.2285,  0.3068],
       device='cuda:0')

In [22]:

quantizer.deployment_model.model[0].weight.data

tensor([[[[ 0.0109,  0.0000, -0.0055],
          [ 0.0109, -0.0000, -0.0073],
          [ 0.0109,  0.0000, -0.0073]],

         [[ 0.0000, -0.0036, -0.0036],
          [ 0.0073,  0.0000, -0.0091],
          [ 0.0128,  0.0055,  0.0000]],

         [[-0.0273, -0.0292, -0.0255],
          [-0.0219, -0.0219, -0.0292],
          [-0.0201, -0.0164, -0.0164]]],


        [[[ 0.0055,  0.0729,  0.0474],
          [-0.0182, -0.0018,  0.0036],
          [-0.0456, -0.1130, -0.0547]],

         [[ 0.1039,  0.1823,  0.1294],
          [-0.0219, -0.0292,  0.0073],
          [-0.1659, -0.2826, -0.1513]],

         [[ 0.0018,  0.0547,  0.0401],
          [-0.0036,  0.0073,  0.0146],
          [-0.0438, -0.0674, -0.0182]]],


        [[[ 0.0201,  0.0438,  0.0346],
          [ 0.0201,  0.0219, -0.0018],
          [ 0.0000,  0.0164,  0.0055]],

         [[-0.0109,  0.0128, -0.0073],
          [-0.0036, -0.0109, -0.0255],
          [-0.0164, -0.0292, -0.0365]],

         [[-0.0273, -0.0292, -0.0310],
     

In [16]:
val_loss, val_prec1, val_prec5 = validate(val_loader, quantizer.deployment_model, criterion, 0, quantizer, gpus)
print(val_loss, val_prec1, val_prec5)

EVALUATING - Epoch: [0][0/782]	Time 5.019 (5.019)	Data 3.639 (3.639)	Loss 8.2795 (8.2795)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][10/782]	Time 0.965 (1.337)	Data 0.000 (0.331)	Loss 8.7712 (9.9373)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][20/782]	Time 0.964 (1.161)	Data 0.000 (0.174)	Loss 10.0164 (9.0190)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][30/782]	Time 0.952 (1.098)	Data 0.000 (0.118)	Loss 8.3803 (9.0293)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][40/782]	Time 0.968 (1.065)	Data 0.000 (0.089)	Loss 7.8637 (9.0348)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][50/782]	Time 0.975 (1.043)	Data 0.000 (0.072)	Loss 8.8594 (9.0806)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][60/782]	Time 0.970 (1.029)	Data 0.000 (0.060)	Loss 8.8920 (9.0228)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
EVALUATING - Epoch: [0][70/782]	Time 0.964 (1.020)	Data 0.000 (0.052)

EVALUATING - Epoch: [0][620/782]	Time 0.993 (0.966)	Data 0.000 (0.006)	Loss 7.8771 (7.7980)	Prec@1 0.000 (0.126)	Prec@5 0.000 (0.599)
EVALUATING - Epoch: [0][630/782]	Time 0.971 (0.965)	Data 0.000 (0.006)	Loss 7.7878 (7.7941)	Prec@1 0.000 (0.124)	Prec@5 0.000 (0.589)
EVALUATING - Epoch: [0][640/782]	Time 0.976 (0.965)	Data 0.000 (0.006)	Loss 6.3292 (7.7890)	Prec@1 0.000 (0.122)	Prec@5 0.000 (0.580)
EVALUATING - Epoch: [0][650/782]	Time 0.962 (0.965)	Data 0.000 (0.006)	Loss 8.6536 (7.7830)	Prec@1 0.000 (0.120)	Prec@5 0.000 (0.595)
EVALUATING - Epoch: [0][660/782]	Time 0.961 (0.965)	Data 0.000 (0.006)	Loss 5.8704 (7.7698)	Prec@1 0.000 (0.118)	Prec@5 0.000 (0.586)
EVALUATING - Epoch: [0][670/782]	Time 0.951 (0.965)	Data 0.000 (0.006)	Loss 7.5885 (7.7697)	Prec@1 0.000 (0.116)	Prec@5 0.000 (0.577)
EVALUATING - Epoch: [0][680/782]	Time 0.848 (0.965)	Data 0.000 (0.006)	Loss 8.5749 (7.7752)	Prec@1 0.000 (0.115)	Prec@5 0.000 (0.569)
EVALUATING - Epoch: [0][690/782]	Time 0.965 (0.965)	Data 0.000

7.839918360290527 0.1 0.496


In [114]:
def has_children(module):
    try:
        next(module.children())
        return True
    except StopIteration:
        return False

for module_full_name, module in model.named_modules():
    if has_children(module) is False:
        if type(module) in [nn.Conv2d, nn.Linear]:
            print(module_full_name)


model.0.0
model.1.0
model.1.3
model.2.0
model.2.3
model.3.0
model.3.3
model.4.0
model.4.3
model.5.0
model.5.3
model.6.0
model.6.3
model.7.0
model.7.3
model.8.0
model.8.3
model.9.0
model.9.3
model.10.0
model.10.3
model.11.0
model.11.3
model.12.0
model.12.3
model.13.0
model.13.3
fc


# Quantized Network

## BatchNorm Folding

In [15]:
deployment_model = None
deployment_modules = []
conv_module = []
batch_module = []
act_module = []


def batch_fold(conv_layer, batch_layer):
    weight_tensor = conv_layer.weight.data
    weight_tensor_size = weight_tensor.size()

    eps = batch_layer.eps
    gamma_tensor = batch_layer.weight.data 
    beta_tensor = batch_layer.bias.data 
    mu_tensor = batch_layer.running_mean
    var_tensor = batch_layer.running_var
    
    if conv_layer.bias is None:
        bias_tensor = - mu_tensor 
    else:
        bias_tensor = conv_layer.bias.data - mu_tensor 

    #folded weight
    for n_out_channel in range(weight_tensor_size[0]):
        gamma = gamma_tensor[n_out_channel]
        sigma = math.sqrt(var_tensor[n_out_channel] + eps)
        beta = beta_tensor[n_out_channel]
        
        weight_per_channel = weight_tensor[n_out_channel]
        weight_per_channel =(weight_per_channel*gamma)/sigma
        weight_tensor[n_out_channel] =  weight_per_channel
        
        bias = bias_tensor[n_out_channel]
        bias = ((bias*gamma)/sigma)+beta
        bias_tensor[n_out_channel] = bias
        
    conv_layer.bias = nn.Parameter(bias_tensor)
    return conv_layer


def find_layers(model):
    for child in model.children():
        if isinstance(child, nn.Sequential):
            find_layers(child) 
        elif isinstance(child, nn.Conv2d):
            conv_module.append(child)
        elif isinstance(child, nn.BatchNorm2d):
            batch_module.append(child)
        elif isinstance(child, nn.ReLU):
            act_module.append(child)
        else:
            print('Other module type')
            deployment_modules.append(child)

            
        #print(child.__class__.__name__)
        if len(conv_module) > 0 and len(batch_module) > 0 and len(act_module) > 0 :
            conv_layer = copy.deepcopy(conv_module.pop())
            batch_layer = copy.deepcopy(batch_module.pop() )
            act_layer = copy.deepcopy(act_module.pop() )
            conv_layer = batch_fold(conv_layer, batch_layer)
            deployment_modules.append(conv_layer )
            #deployment_modules.append(batch_layer )
            deployment_modules.append( act_layer )
        else:
            pass
        
        
        
find_layers(model.model)
deployment_model = copy.deepcopy(model)
deployment_model.model = nn.Sequential(*deployment_modules)    
deployment_model  


Other module type


mobilenet_real(
  (model): Sequential(
    (0): Conv2d (3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
    (3): ReLU(inplace)
    (4): Conv2d (32, 64, kernel_size=(1, 1), stride=(1, 1))
    (5): ReLU(inplace)
    (6): Conv2d (64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
    (7): ReLU(inplace)
    (8): Conv2d (64, 128, kernel_size=(1, 1), stride=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
    (11): ReLU(inplace)
    (12): Conv2d (128, 128, kernel_size=(1, 1), stride=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
    (15): ReLU(inplace)
    (16): Conv2d (128, 256, kernel_size=(1, 1), stride=(1, 1))
    (17): ReLU(inplace)
    (18): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(

In [16]:
val_loss, val_prec1, val_prec5 = validate(val_loader, deployment_model.cuda(), criterion, 0, None, gpus)
print(val_loss, val_prec1, val_prec5)

EVALUATING - Epoch: [0][0/391]	Time 9.409 (9.409)	Data 2.844 (2.844)	Loss 0.6119 (0.6119)	Prec@1 82.812 (82.812)	Prec@5 96.094 (96.094)
EVALUATING - Epoch: [0][10/391]	Time 0.047 (0.900)	Data 0.000 (0.259)	Loss 1.1026 (0.6558)	Prec@1 72.656 (82.812)	Prec@5 91.406 (95.455)
EVALUATING - Epoch: [0][20/391]	Time 0.050 (0.564)	Data 0.000 (0.206)	Loss 0.9791 (0.9172)	Prec@1 78.906 (76.228)	Prec@5 92.188 (93.192)
EVALUATING - Epoch: [0][30/391]	Time 0.046 (0.455)	Data 0.000 (0.197)	Loss 1.5945 (1.0715)	Prec@1 73.438 (72.681)	Prec@5 86.719 (92.011)
EVALUATING - Epoch: [0][40/391]	Time 0.049 (0.395)	Data 0.000 (0.188)	Loss 0.9665 (0.9725)	Prec@1 76.562 (75.248)	Prec@5 92.969 (92.683)
EVALUATING - Epoch: [0][50/391]	Time 0.300 (0.395)	Data 0.243 (0.219)	Loss 0.3795 (0.9922)	Prec@1 85.938 (75.107)	Prec@5 99.219 (92.341)
EVALUATING - Epoch: [0][60/391]	Time 0.180 (0.366)	Data 0.131 (0.211)	Loss 1.0166 (0.9407)	Prec@1 75.000 (76.422)	Prec@5 91.406 (92.649)
EVALUATING - Epoch: [0][70/391]	Time 0.047

1.3458160171508788 67.652 87.878


# Post Training Quantization of weights as
https://github.com/ARM-software/ML-KWS-for-MCU/blob/master/quant_test.py

https://github.com/ARM-software/ML-KWS-for-MCU/blob/master/quant_models.py

In [17]:
def fixed_point_linear_quant(min_value, max_value, n_bits, signed=True):
    range_values = max_value- min_value

    max_range = max(abs(min_value),abs(max_value))
    int_bits = int(np.ceil(np.log2(max_range)))
    
    if int_bits < 0:
        int_bits = 0
    elif int_bits>(n_bits-1):
        int_bits = (n_bits-1)
        
    if signed:
        frac_bits = n_bits-1-int_bits #remaining bits are fractional bits (1-bit for sign)
    else:
        frac_bits = n_bits-int_bits
        
    return int_bits, frac_bits

In [19]:
stats_conv2d = []
N_BITS = 8
def f_quant_weight(model, symmetric_range=True):
    if symmetric_range is not True: # asymmetric to be implemented
        print('Error')
        return -1
    for child in model.children():
        if isinstance(child, nn.Sequential):
            f_quant_weight(child, symmetric_range) 
        elif isinstance(child, nn.Conv2d) or isinstance(child, nn.Linear):
            max_weights = child.weight.data.max()
            min_weights = child.weight.data.min()
            range_weights = max_weights- min_weights
            int_bits, frac_bits = fixed_point_linear_quant(min_weights, max_weights, N_BITS, signed=True)
            
#            max_range = max(abs(min_weights),abs(max_weights))
#            int_bits = int(np.ceil(np.log2(max_range)))
#            if int_bits<0:
#                int_bits = 0
#            elif int_bits>(N_BITS-1):
#                int_bits = (N_BITS-1)
#            frac_bits = N_BITS-1-int_bits #remaining bits are fractional bits (1-bit for sign)           
            
            stats_conv2d.append([max_weights, min_weights, range_weights, int_bits, frac_bits])
            print(max_weights, min_weights, range_weights, int_bits, frac_bits)
            if int_bits > 0:
                max_value = (2**(int_bits-1))+1-2**(-frac_bits)
            else:
                max_value = 1-2**(-frac_bits)
            #print(int_bits, -2**int_bits, (2**int_bits)-1, max_value, 2**(-frac_bits))
            clipped_weight = child.weight.data.clamp( -max_value, max_value)
            quant_weight = clipped_weight.mul(2**frac_bits).round().div(2**frac_bits)
            #print(quant_weight)
            child.weight.data = quant_weight
            
            
f_quant_weight(deployment_model, symmetric_range=True)
stats_conv2d

0.558258593082428 -0.8026384711265564 1.3608970642089844 0 7
4.027159690856934 -4.618956565856934 8.646116256713867 3 4
1.2754887342453003 -1.4489389657974243 2.7244277000427246 1 6
5.171591758728027 -3.461690664291382 8.63328242301941 3 4
0.8815251588821411 -0.831538736820221 1.713063895702362 0 7
4.645492076873779 -4.746695041656494 9.392187118530273 3 4
0.9018105864524841 -0.8606885075569153 1.7624990940093994 0 7
2.1124305725097656 -2.2831485271453857 4.395579099655151 2 5
0.571179986000061 -0.4470636546611786 1.0182436406612396 0 7
6.458991527557373 -5.2962141036987305 11.755205631256104 3 4
0.5835903286933899 -0.5659104585647583 1.1495007872581482 0 7
22.159364700317383 -2.932857036590576 25.09222173690796 5 2
0.3646238148212433 -0.40404146909713745 0.7686652839183807 0 7
19.831918716430664 -6.3695831298828125 26.201501846313477 5 2
0.45581403374671936 -0.3711283206939697 0.8269423544406891 0 7
9.509058952331543 -5.71013879776001 15.219197750091553 4 3
0.401159405708313 -0.382216

[[0.558258593082428, -0.8026384711265564, 1.3608970642089844, 0, 7],
 [4.027159690856934, -4.618956565856934, 8.646116256713867, 3, 4],
 [1.2754887342453003, -1.4489389657974243, 2.7244277000427246, 1, 6],
 [5.171591758728027, -3.461690664291382, 8.63328242301941, 3, 4],
 [0.8815251588821411, -0.831538736820221, 1.713063895702362, 0, 7],
 [4.645492076873779, -4.746695041656494, 9.392187118530273, 3, 4],
 [0.9018105864524841, -0.8606885075569153, 1.7624990940093994, 0, 7],
 [2.1124305725097656, -2.2831485271453857, 4.395579099655151, 2, 5],
 [0.571179986000061, -0.4470636546611786, 1.0182436406612396, 0, 7],
 [6.458991527557373, -5.2962141036987305, 11.755205631256104, 3, 4],
 [0.5835903286933899, -0.5659104585647583, 1.1495007872581482, 0, 7],
 [22.159364700317383, -2.932857036590576, 25.09222173690796, 5, 2],
 [0.3646238148212433, -0.40404146909713745, 0.7686652839183807, 0, 7],
 [19.831918716430664, -6.3695831298828125, 26.201501846313477, 5, 2],
 [0.45581403374671936, -0.37112832069

# Post Training Quantization of activations

In [20]:
output_range_min =  []
output_range_max =  []
output_range_mean =  []
output_range_std =  []
output_range_values = []
def get_io_shape_3(self, input, output):
    output_range_min[self.mark].append(output.data.min())
    output_range_max[self.mark].append(output.data.max())
    output_range_mean[self.mark].append(output.data.mean())
    output_range_std[self.mark].append(output.data.std())
    print(output.data.max(),output.data.min(),output.data.mean(),output.data.std())

    
    
def calibrate(data_loader, model, n_batch, gpus):
    training = False
    
    counter = 0
    for n,l in model.named_modules():
        if isinstance(l,nn.ReLU):
            print(n,l)
            l.mark = counter
            output_range_min.append([])
            output_range_max.append([])
            output_range_std.append([])
            output_range_mean.append([])
            counter += 1 
            l.register_forward_hook(get_io_shape_3)
        
    #v = model.model[1].register_forward_hook(get_layer8)

    #if gpus and len(gpus) > 1:
    #    model = torch.nn.DataParallel(model, gpus)
    model.eval()
    for i, (inputs, target) in enumerate(data_loader):
        if i >= n_batch:
            break
            
        input_var = Variable(inputs.type(ttype), volatile=not training)
        target_var = Variable(target)
        
        # compute output
        output = model(input_var)
        
        #print(v) 

    return output


# Run validation
output = calibrate(train_loader, deployment_model.cuda(), 10, gpus)
print(output)


model.1 ReLU(inplace)
model.3 ReLU(inplace)
model.5 ReLU(inplace)
model.7 ReLU(inplace)
model.9 ReLU(inplace)
model.11 ReLU(inplace)
model.13 ReLU(inplace)
model.15 ReLU(inplace)
model.17 ReLU(inplace)
model.19 ReLU(inplace)
model.21 ReLU(inplace)
model.23 ReLU(inplace)
model.25 ReLU(inplace)
model.27 ReLU(inplace)
model.29 ReLU(inplace)
model.31 ReLU(inplace)
model.33 ReLU(inplace)
model.35 ReLU(inplace)
model.37 ReLU(inplace)
model.39 ReLU(inplace)
model.41 ReLU(inplace)
model.43 ReLU(inplace)
model.45 ReLU(inplace)
model.47 ReLU(inplace)
model.49 ReLU(inplace)
model.51 ReLU(inplace)
model.53 ReLU(inplace)
5.781166076660156 0.0 0.3834649622440338 0.4645973742008209
11.030117988586426 0.0 0.4068128168582916 0.5873093605041504
15.173482894897461 0.0 0.29163435101509094 0.34446993470191956
9.82027530670166 0.0 0.38282033801078796 0.4403855800628662
7.053549289703369 0.0 0.24418221414089203 0.2565935552120209
8.497363090515137 0.0 0.31330856680870056 0.41559818387031555
6.900217056274414

7.011906147003174 0.0 0.24467183649539948 0.25533583760261536
9.05483627319336 0.0 0.31126174330711365 0.41254761815071106
7.086294651031494 0.0 0.16814205050468445 0.2536892890930176
7.003238201141357 0.0 0.36242273449897766 0.40484511852264404
4.015329837799072 0.0 0.2347562611103058 0.22057360410690308
5.846207618713379 0.0 0.2059193253517151 0.3024962544441223
3.634432792663574 0.0 0.1273060292005539 0.194541797041893
7.402890205383301 0.0 0.25346988439559937 0.3365992605686188
2.5735719203948975 0.0 0.17581391334533691 0.19792959094047546
5.480377674102783 0.0 0.16607365012168884 0.24425289034843445
4.021682262420654 0.0 0.11848624795675278 0.16565629839897156
6.608579158782959 0.0 0.18902117013931274 0.27147623896598816
2.855058431625366 0.0 0.09935259073972702 0.15912334620952606
7.009454250335693 0.0 0.1912030577659607 0.2737687826156616
2.9869165420532227 0.0 0.0953017920255661 0.16059397161006927
11.047274589538574 0.0 0.21059156954288483 0.2868184745311737
3.5444271564483643

In [29]:
mean(output_range_min[0])

NameError: name 'mean' is not defined

## Determine precision per layer

In [30]:
n_act_layer = len(output_range_min)
precision_layer = [[] for x in range(n_act_layer)]

In [33]:
for i in range(n_act_layer):
    min_value = min(output_range_min[i])
    max_value = max(output_range_max[i])
    std_value = max(output_range_std[i])
    int_bits, frac_bits = fixed_point_linear_quant(min_value, 3*std_value, N_BITS, signed=False)
    print(int_bits, frac_bits)

1 7
1 7
1 7
1 7
0 8
1 7
0 8
1 7
0 8
0 8
0 8
1 7
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
0 8
2 6


In [93]:
def QuantizeFixed(real_activation, numBits=8, min_value=0, max_value=6, int_bits=1):
    tensor = real_activation.clamp( 0, max_value )
    eps = (max_value)/((2**numBits)-1)
    shift_int = 2**int_bits
    tensor=tensor.div(eps).floor().mul(eps)
    #tensor = tensor.div(shift_int)
    #print(eps,shift_int )
    return tensor


class QuantizationReLU(torch.autograd.Function):
    '''
    Quantize the input activations and calculate the mean across channel dimension.
    '''
    @staticmethod
    def forward(ctx, input, bits, min_value, max_value, int_bits):
        ctx.save_for_backward(input)
        output = QuantizeFixed(input, numBits=bits, min_value=min_value, max_value=max_value, int_bits=int_bits)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input.ge(1)] = 0
        grad_input[input.le(0)] = 0
        return grad_input, None, None, None, None


class QuantReLU(nn.Module):
    def __init__(self, min_value, max_value, num_of_bits, int_bits):
        super(QuantReLU,self).__init__()
        self.bits = num_of_bits
        self.max_value = max_value
        self.min_value = min_value
        self.int_bits = int_bits
        
    def forward(self,input):
        return QuantizationReLU.apply(input, self.bits, self.min_value,self.max_value, self.int_bits )

# Replacing Activations

In [98]:
quantized_model = None
quantized_modules = []
counter = 0

def find_activations_layers(model):
    for child in model.children():
        if isinstance(child, nn.Sequential):
            find_activations_layers(child) 
        elif isinstance(child, nn.ReLU):
            global counter
            min_value = min(output_range_min[counter])
            max_value = max(output_range_max[counter])
            std_value = max(output_range_std[counter])
            counter += 1
            max_value = 7*std_value
            print(max_value)
            int_bits, frac_bits = fixed_point_linear_quant(min_value, max_value, N_BITS, signed=False)
            quantized_modules.append(QuantReLU(min_value, max_value, N_BITS, int_bits ) )
        else:
            quantized_modules.append(child)
            
            
find_activations_layers(deployment_model.model)
quantized_model = copy.deepcopy(deployment_model)
quantized_model.model = nn.Sequential(*quantized_modules)    
quantized_model  

3.2969404458999634
4.1698092222213745
2.42896831035614
3.1248030364513397
1.8195737302303314
2.9339458644390106
1.8078442811965942
2.8597283363342285
1.5590582340955734
2.136590749025345
1.3774169981479645
2.384040504693985
1.4072715491056442
1.729140356183052
1.1710454523563385
1.910538762807846
1.121280461549759
1.924646645784378
1.1285786926746368
2.010012626647949
1.0860780328512192
2.113189846277237
1.0360946208238602
2.2392912805080414
0.753291554749012
1.8327347040176392
7.726824045181274


mobilenet_real(
  (model): Sequential(
    (0): Conv2d (3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): QuantReLU(
    )
    (2): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
    (3): QuantReLU(
    )
    (4): Conv2d (32, 64, kernel_size=(1, 1), stride=(1, 1))
    (5): QuantReLU(
    )
    (6): Conv2d (64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
    (7): QuantReLU(
    )
    (8): Conv2d (64, 128, kernel_size=(1, 1), stride=(1, 1))
    (9): QuantReLU(
    )
    (10): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
    (11): QuantReLU(
    )
    (12): Conv2d (128, 128, kernel_size=(1, 1), stride=(1, 1))
    (13): QuantReLU(
    )
    (14): Conv2d (128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
    (15): QuantReLU(
    )
    (16): Conv2d (128, 256, kernel_size=(1, 1), stride=(1, 1))
    (17): QuantReLU(
    )
    (18): Conv2d (256, 256, kernel_size=(3, 3

In [99]:
val_loss, val_prec1, val_prec5 = validate(val_loader, model, criterion, 0, None, gpus)
print(val_loss, val_prec1, val_prec5)

EVALUATING - Epoch: [0][0/391]	Time 3.287 (3.287)	Data 3.211 (3.211)	Loss 0.6898 (0.6898)	Prec@1 83.594 (83.594)	Prec@5 92.188 (92.188)
EVALUATING - Epoch: [0][10/391]	Time 0.542 (0.509)	Data 0.473 (0.440)	Loss 1.3616 (0.9342)	Prec@1 64.062 (76.989)	Prec@5 85.938 (91.051)
EVALUATING - Epoch: [0][20/391]	Time 0.213 (0.357)	Data 0.143 (0.288)	Loss 1.5703 (1.1213)	Prec@1 64.844 (71.429)	Prec@5 82.812 (90.141)
EVALUATING - Epoch: [0][30/391]	Time 0.067 (0.327)	Data 0.000 (0.257)	Loss 1.8310 (1.2872)	Prec@1 66.406 (67.288)	Prec@5 82.031 (88.659)
EVALUATING - Epoch: [0][40/391]	Time 0.232 (0.303)	Data 0.160 (0.234)	Loss 1.0482 (1.1945)	Prec@1 74.219 (69.798)	Prec@5 89.844 (89.348)
EVALUATING - Epoch: [0][50/391]	Time 0.070 (0.300)	Data 0.000 (0.230)	Loss 0.6157 (1.2641)	Prec@1 82.812 (68.903)	Prec@5 94.531 (88.251)
EVALUATING - Epoch: [0][60/391]	Time 0.310 (0.288)	Data 0.240 (0.219)	Loss 1.2655 (1.2117)	Prec@1 69.531 (70.338)	Prec@5 87.500 (88.781)
EVALUATING - Epoch: [0][70/391]	Time 0.144

1.5934641359710693 62.202 84.306


# ---------- Extra ----------- 

In [43]:
a = np.random.rand(5)
np.clip(a,0,0.5)

array([ 0.3250339 ,  0.5       ,  0.21670543,  0.23417938,  0.5       ])

In [61]:
batch_fold_vector = [
['module.model.0.0','module.model.0.1'],
['module.model.1.0','module.model.1.1'],
['module.model.1.3','module.model.1.4'],
['module.model.2.0','module.model.2.1'],
['module.model.2.3','module.model.2.4'],
['module.model.3.0','module.model.3.1'],
['module.model.3.3','module.model.3.4'],
['module.model.4.0','module.model.4.1'],
['module.model.4.3','module.model.4.4'],
['module.model.5.0','module.model.5.1'],
['module.model.5.3','module.model.5.4'],
['module.model.6.0','module.model.6.1'],
['module.model.6.3','module.model.6.4'],
['module.model.7.0','module.model.7.1'],
['module.model.7.3','module.model.7.4'],
['module.model.8.0','module.model.8.1'],
['module.model.8.3','module.model.8.4'],
['module.model.9.0','module.model.9.1'],
['module.model.9.3','module.model.9.4'],
['module.model.10.0','module.model.10.1'],
['module.model.10.3','module.model.10.4'],
['module.model.11.0','module.model.11.1'],
['module.model.11.3','module.model.11.4'],
['module.model.12.0','module.model.12.1'],
['module.model.12.3','module.model.12.4'],
['module.model.13.0','module.model.13.1'],
['module.model.13.3','module.model.13.4']
]
    

torch.Size([32, 3, 3, 3])
torch.Size([32, 1, 3, 3])
torch.Size([64, 32, 1, 1])
torch.Size([64, 1, 3, 3])
torch.Size([128, 64, 1, 1])
torch.Size([128, 1, 3, 3])
torch.Size([128, 128, 1, 1])
torch.Size([128, 1, 3, 3])
torch.Size([256, 128, 1, 1])
torch.Size([256, 1, 3, 3])
torch.Size([256, 256, 1, 1])
torch.Size([256, 1, 3, 3])
torch.Size([512, 256, 1, 1])
torch.Size([512, 1, 3, 3])
torch.Size([512, 512, 1, 1])
torch.Size([512, 1, 3, 3])
torch.Size([512, 512, 1, 1])
torch.Size([512, 1, 3, 3])
torch.Size([512, 512, 1, 1])
torch.Size([512, 1, 3, 3])
torch.Size([512, 512, 1, 1])
torch.Size([512, 1, 3, 3])
torch.Size([512, 512, 1, 1])
torch.Size([512, 1, 3, 3])
torch.Size([1024, 512, 1, 1])
torch.Size([1024, 1, 3, 3])
torch.Size([1024, 1024, 1, 1])


In [63]:
weight_tensor_size[0]

1024

In [14]:
conv.quant_weight

True

In [16]:
index = -1
index_conv2d = []
index_lin = []

for idx,m in enumerate(model.modules()):
    print('Index = ', idx)
    weight_quant = getattr(m, 'quant_weight', None)
    if weight_quant != None:
        if isinstance(m, nn.Conv2d):
            index = index + 1
            index_conv2d.append(index)
            print('Conv module ', idx, 'quantized!')
        elif isinstance(m, nn.Linear):
            index = index + 1
            index_lin.append(index)
            print('Linear module: ', idx, ' quantized!')
        else: 
            print('Module: ', idx, ' NOT quantized!')
        
print('************************')
print('Tot to quantized: ', len(index_conv2d) + len(index_lin))

Index =  0
Index =  1
Index =  2
Conv module  2 quantized!
Index =  3
Index =  4
Index =  5
Index =  6
Index =  7
Conv module  7 quantized!
Index =  8
Index =  9
Index =  10
Index =  11
Index =  12
Conv module  12 quantized!
Index =  13
Index =  14
Index =  15
Index =  16
Conv module  16 quantized!
Index =  17
Index =  18
Index =  19
Index =  20
Conv module  20 quantized!
Index =  21
Index =  22
Index =  23
Index =  24
Index =  25
Linear module:  25  quantized!
Index =  26
Index =  27
Index =  28
Linear module:  28  quantized!
Index =  29
Index =  30
Index =  31
Linear module:  31  quantized!
Index =  32
************************
Tot to quantized:  8


In [6]:
conv = model.features[0]

In [10]:
model.features[1].quant_weight

AttributeError: 'MaxPool2d' object has no attribute 'quant_weight'

In [19]:
#trianing 
epoch = 0
model.train()
optimizer = adjust_optimizer(optimizer, epoch, regime)
    

In [20]:
optimizer

In [155]:
inputs,target = next(iter(train_loader))

In [156]:
input_var = Variable(inputs, volatile=False)
target_var = Variable(target)


In [164]:
quantizer.binarization()

In [167]:
output = model(input_var)
loss = criterion(output, target_var)


In [169]:
optimizer.zero_grad()

In [171]:
loss.backward()

In [161]:
quantizer.restore()

In [184]:
optimizer.step()

In [174]:
for index in range(quantizer.num_of_params):
    quantizer.target_modules[index].data.copy_(quantizer.saved_params[index])

In [179]:
quantizer.updateBinaryGradWeight2()

In [183]:
conv1.weight.data


( 0 , 0 ,.,.) = 
  1.8127e-02  1.5027e-03  3.2594e-03 -1.2902e-02  1.4510e-02
  7.1123e-03  1.9530e-02 -1.0884e-02 -2.8592e-03 -1.3259e-02
  2.1212e-02 -2.1252e-02 -8.8512e-03  5.5863e-03  2.5938e-02
  2.4559e-02 -1.9748e-02  2.2135e-02  1.5002e-03 -1.0777e-02
 -1.0942e-02  3.0251e-03 -8.2488e-03 -2.0687e-02 -3.1814e-03

( 0 , 1 ,.,.) = 
 -2.3760e-03 -2.6540e-03 -1.1723e-02 -1.5072e-02 -1.2241e-03
 -1.3689e-02 -2.1758e-02  1.1396e-02  2.5437e-02 -8.7456e-04
  2.2533e-03 -1.3677e-02 -5.6864e-03  2.1138e-02  4.9958e-03
  1.5956e-03 -3.7952e-03 -6.2803e-03  1.3870e-02  1.3949e-02
  1.2288e-02  1.0312e-02 -8.1034e-04  1.1017e-02 -1.1564e-02

( 0 , 2 ,.,.) = 
 -9.1109e-03 -9.9038e-04  6.6380e-03 -2.5215e-02 -1.5876e-03
 -1.5147e-02 -3.6442e-03  1.6910e-02  4.4561e-04 -1.8616e-02
 -2.3990e-02  2.0563e-03 -2.4307e-02 -8.7227e-03  1.0326e-02
  2.3016e-02 -2.1399e-02  2.1827e-02  1.4622e-02 -1.6246e-02
 -1.4855e-02 -4.9985e-03 -1.8892e-02 -1.4951e-03  4.6154e-03
    ... 

( 0 ,61 ,.,.) = 
 -7.

In [181]:
conv1.weight.grad.data


( 0 , 0 ,.,.) = 
 -0.0305 -0.0396 -0.0279 -0.0134 -0.0152
 -0.0114 -0.0266 -0.0116 -0.0244 -0.0115
 -0.0183 -0.0288 -0.0244 -0.0335 -0.0331
  0.0048  0.0138 -0.0234 -0.0050 -0.0150
 -0.0089  0.0039  0.0133  0.0012 -0.0098

( 0 , 1 ,.,.) = 
 -0.0149 -0.0453 -0.0115  0.0112  0.0002
 -0.0263 -0.0172 -0.0210  0.0020 -0.0010
 -0.0262 -0.0166 -0.0302 -0.0196 -0.0029
 -0.0282 -0.0147 -0.0169 -0.0244 -0.0162
 -0.0065 -0.0139 -0.0212  0.0036  0.0018

( 0 , 2 ,.,.) = 
 -0.0039 -0.0094 -0.0027  0.0085  0.0162
 -0.0215 -0.0199  0.0074 -0.0156  0.0015
 -0.0125 -0.0276 -0.0038 -0.0135 -0.0058
  0.0034  0.0195 -0.0195  0.0120 -0.0050
 -0.0144  0.0106  0.0295  0.0133  0.0012
    ... 

( 0 ,61 ,.,.) = 
 -0.0010 -0.0220 -0.0113 -0.0068 -0.0008
 -0.0075 -0.0106 -0.0159  0.0005 -0.0021
 -0.0197 -0.0172 -0.0132 -0.0122  0.0000
 -0.0088 -0.0091 -0.0031 -0.0062  0.0006
  0.0066 -0.0009 -0.0075 -0.0113 -0.0007

( 0 ,62 ,.,.) = 
  0.0064  0.0128 -0.0052 -0.0077 -0.0012
 -0.0130 -0.0098 -0.0072  0.0094 -0.0054

In [185]:
conv1.weight.data


( 0 , 0 ,.,.) = 
  1.8670e-02  2.2519e-03  4.0718e-03 -1.2053e-02  1.5346e-02
  7.2466e-03  2.0200e-02 -1.0133e-02 -2.1329e-03 -1.2672e-02
  2.1838e-02 -2.0518e-02 -8.0581e-03  6.3540e-03  2.6678e-02
  2.4403e-02 -2.0051e-02  2.2867e-02  2.3579e-03 -9.9237e-03
 -1.0284e-02  2.7908e-03 -8.8978e-03 -2.0199e-02 -2.3274e-03

( 0 , 1 ,.,.) = 
 -1.7704e-03 -1.8924e-03 -1.0922e-02 -1.5883e-02 -1.8224e-03
 -1.2829e-02 -2.0913e-02  1.2217e-02  2.5780e-02 -1.2984e-04
  2.9267e-03 -1.2853e-02 -5.1088e-03  2.1482e-02  5.8533e-03
  2.2906e-03 -3.0871e-03 -5.7004e-03  1.4502e-02  1.4483e-02
  1.2501e-02  1.0770e-02 -1.9770e-04  1.1162e-02 -1.2295e-02

( 0 , 2 ,.,.) = 
 -8.3275e-03 -1.1672e-03  6.1495e-03 -2.6007e-02 -2.4467e-03
 -1.4519e-02 -3.3815e-03  1.6122e-02  8.5915e-04 -1.9005e-02
 -2.3674e-02  2.4657e-03 -2.3547e-02 -7.8640e-03  1.0768e-02
  2.3381e-02 -2.2082e-02  2.2670e-02  1.4836e-02 -1.5474e-02
 -1.4576e-02 -5.6797e-03 -1.9072e-02 -2.1535e-03  4.0057e-03
    ... 

( 0 ,61 ,.,.) = 
 -7.

In [144]:
quantizer.binarization()

In [145]:
conv1.weight.data


( 0 , 0 ,.,.) = 
  1  1  1 -1  1
  1  1 -1 -1 -1
  1 -1 -1  1  1
  1 -1  1  1 -1
 -1  1 -1 -1 -1

( 0 , 1 ,.,.) = 
 -1 -1 -1 -1 -1
 -1 -1  1  1 -1
  1 -1 -1  1  1
  1 -1 -1  1  1
  1  1 -1  1 -1

( 0 , 2 ,.,.) = 
 -1 -1  1 -1 -1
 -1 -1  1  1 -1
 -1  1 -1 -1  1
  1 -1  1  1 -1
 -1 -1 -1 -1  1
    ... 

( 0 ,61 ,.,.) = 
 -1  1  1  1 -1
 -1 -1  1 -1 -1
  1 -1 -1  1 -1
  1 -1 -1  1  1
 -1 -1  1 -1 -1

( 0 ,62 ,.,.) = 
  1 -1 -1  1 -1
 -1 -1  1  1  1
 -1 -1 -1  1  1
  1 -1 -1 -1 -1
 -1 -1  1 -1 -1

( 0 ,63 ,.,.) = 
 -1 -1  1 -1 -1
 -1  1 -1  1 -1
  1 -1 -1 -1  1
 -1  1 -1  1 -1
 -1 -1 -1 -1  1
      ⋮  

( 1 , 0 ,.,.) = 
  1 -1  1 -1  1
  1 -1 -1  1  1
  1  1  1  1  1
 -1  1  1  1 -1
 -1 -1  1  1  1

( 1 , 1 ,.,.) = 
  1  1 -1 -1 -1
  1 -1  1  1 -1
  1 -1  1  1  1
  1 -1 -1  1  1
  1 -1 -1 -1  1

( 1 , 2 ,.,.) = 
 -1  1  1  1  1
 -1  1  1 -1 -1
 -1 -1  1 -1 -1
  1  1 -1  1 -1
  1  1 -1  1 -1
    ... 

( 1 ,61 ,.,.) = 
  1  1 -1  1  1
 -1  1 -1  1  1
 -1  1  1 -1  1
 -1 -1 -1 -1  1
 -1  1  

In [146]:
output = model(input_var)

In [148]:
quantizer.restore()

In [149]:
conv1.weight.data


( 0 , 0 ,.,.) = 
  1.8873e-02  7.5993e-04  2.5151e-03 -1.3646e-02  1.3772e-02
  7.8605e-03  1.8820e-02 -1.1640e-02 -3.6067e-03 -1.2481e-02
  2.2044e-02 -2.1994e-02 -9.5977e-03  4.8411e-03  2.5192e-02
  2.3820e-02 -2.0491e-02  2.1387e-02  7.5171e-04 -1.1523e-02
 -1.1641e-02  2.2801e-03 -7.4655e-03 -2.1435e-02 -3.9254e-03

( 0 , 1 ,.,.) = 
 -1.7088e-03 -3.4021e-03 -1.2472e-02 -1.4329e-02 -4.8686e-04
 -1.4436e-02 -2.2507e-02  1.0649e-02  2.4689e-02 -1.6210e-03
  1.4775e-03 -1.4429e-02 -4.9556e-03  2.1881e-02  4.2458e-03
  8.3148e-04 -4.5646e-03 -5.5588e-03  1.4497e-02  1.4675e-02
  1.3019e-02  1.1046e-02 -9.1878e-05  1.0265e-02 -1.0822e-02

( 0 , 2 ,.,.) = 
 -9.8510e-03 -2.4725e-04  7.3819e-03 -2.4469e-02 -8.3953e-04
 -1.4348e-02 -2.9021e-03  1.7654e-02  1.1927e-03 -1.9339e-02
 -2.3239e-02  2.8028e-03 -2.5049e-02 -9.4663e-03  1.1083e-02
  2.2274e-02 -2.0638e-02  2.1082e-02  1.3878e-02 -1.6996e-02
 -1.4111e-02 -4.2093e-03 -1.9636e-02 -7.8934e-04  3.7102e-03
    ... 

( 0 ,61 ,.,.) = 
 -6.

In [141]:
conv1.weight.grad.data
optimizer.zero_grad()

In [150]:
conv1.weight.grad.data


( 0 , 0 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

( 0 , 1 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

( 0 , 2 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
    ... 

( 0 ,61 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

( 0 ,62 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

( 0 ,63 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
      ⋮  

( 1 , 0 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

( 1 , 1 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

( 1 , 2 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
    ... 

( 1 ,61 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  

In [58]:
for i,v in enumerate(model.named_modules()):
    print(i, '->', v)

0 -> ('', VGG(
  (first_feature): Sequential(
    (0): Conv2d (3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  )
  (last_layer): Sequential(
    (0): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
    (1): BinActive(
    )
    (2): Linear(in_features=4096, out_features=1000)
    (3): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
  )
  (features): Sequential(
    (0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (1): BinConv(
      (ConvLayer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (1): BinActive(
        )
        (2): Conv2d (64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      )
    )
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): BinConv(
      (ConvLayer): Sequential(
        (0): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
        (1): BinActive(
        )
        (2): Conv2d (192, 384, kernel_size=(3, 3), stride=(1, 1), pa

In [120]:
model.eval()

VGG(
  (first_feature): Sequential(
    (0): Conv2d (3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  )
  (last_layer): Sequential(
    (0): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
    (1): BinActive(
    )
    (2): Linear(in_features=4096, out_features=1000)
    (3): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
  )
  (features): Sequential(
    (0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (1): BinConv(
      (ConvLayer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (1): BinActive(
        )
        (2): Conv2d (64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      )
    )
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): BinConv(
      (ConvLayer): Sequential(
        (0): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
        (1): BinActive(
        )
        (2): Conv2d (192, 384, kernel_size=(3, 3), stride=(1, 1), 

In [119]:
conv0 = model.first_feature[0]


In [73]:
binconv0 = model.features[1]
conv1 = binconv0.ConvLayer[2]


In [74]:
model.sta


(0 ,0 ,.,.) = 
  3.2999e-02 -1.0853e-02  1.1804e-02  ...  -4.1577e-02 -4.5362e-02 -3.6687e-02
 -3.4907e-02  4.2962e-02  1.5821e-02  ...   1.2582e-02 -1.1026e-02  2.7164e-02
  3.2744e-02  4.2307e-02 -4.9959e-02  ...   1.2236e-02  3.4432e-02 -2.2982e-02
                 ...                   ⋱                   ...                
 -2.3553e-02 -1.7643e-02  8.8369e-03  ...   1.0509e-02 -4.5603e-03 -1.2793e-02
  3.6100e-02  1.9320e-02 -4.3570e-03  ...   3.2710e-02  4.9394e-02  3.6253e-02
 -1.5289e-02  1.6344e-02 -1.5270e-02  ...  -8.6423e-03 -4.2889e-02  4.5982e-02

(0 ,1 ,.,.) = 
 -2.7279e-02  5.1558e-02 -4.3274e-02  ...   4.1317e-02  1.3197e-02  3.0461e-02
 -3.9469e-02  1.6336e-02 -3.3295e-02  ...  -1.9971e-02  1.6847e-02  1.2991e-02
  3.8012e-02  3.1919e-02  2.9151e-02  ...  -2.0594e-02 -1.5257e-02  1.7365e-02
                 ...                   ⋱                   ...                
  2.5971e-02  4.3248e-02  4.6850e-02  ...   2.4054e-02  3.7525e-02 -3.7133e-02
  4.7485e-02  1.9903

In [99]:
conv1.weight.data


( 0 , 0 ,.,.) = 
  1  1  1 -1  1
  1  1 -1 -1 -1
  1 -1 -1  1  1
  1 -1  1 -1 -1
 -1  1 -1 -1 -1

( 0 , 1 ,.,.) = 
 -1 -1 -1 -1 -1
 -1 -1  1  1 -1
  1 -1 -1  1  1
 -1 -1 -1  1  1
  1  1 -1  1 -1

( 0 , 2 ,.,.) = 
 -1 -1  1 -1  1
 -1 -1  1  1 -1
 -1  1 -1 -1  1
  1 -1  1  1 -1
 -1 -1 -1 -1  1
    ... 

( 0 ,61 ,.,.) = 
 -1  1  1  1 -1
 -1 -1  1 -1 -1
  1 -1 -1  1 -1
  1 -1 -1  1  1
 -1 -1  1 -1 -1

( 0 ,62 ,.,.) = 
  1 -1 -1  1 -1
 -1 -1  1  1  1
  1 -1 -1  1  1
  1 -1 -1 -1 -1
 -1 -1  1 -1 -1

( 0 ,63 ,.,.) = 
 -1 -1  1 -1 -1
 -1  1 -1 -1 -1
 -1 -1 -1  1  1
 -1  1 -1  1 -1
 -1 -1 -1 -1  1
      ⋮  

( 1 , 0 ,.,.) = 
  1 -1  1 -1  1
  1 -1 -1  1  1
  1  1  1 -1  1
 -1  1  1  1 -1
 -1 -1  1  1  1

( 1 , 1 ,.,.) = 
  1  1 -1 -1 -1
  1 -1  1  1 -1
  1 -1  1  1  1
  1 -1 -1  1  1
  1 -1 -1 -1  1

( 1 , 2 ,.,.) = 
 -1  1  1  1  1
 -1  1  1 -1 -1
 -1 -1  1 -1 -1
  1  1 -1  1 -1
  1  1 -1  1 -1
    ... 

( 1 ,61 ,.,.) = 
  1  1 -1  1  1
 -1  1  1  1  1
 -1  1  1 -1  1
 -1 -1 -1 -1  1
 -1  1  

In [76]:
quantizer.binarization()

In [100]:
quantizer.restore()

AttributeError: 'NoneType' object has no attribute 'data'

In [79]:
out0 = model.first_feature(input_var)

In [82]:
maxpoo = model.features[0]

In [83]:
out1 = maxpoo(out0)

In [86]:
batch1 = binconv0.ConvLayer[0]
binact1 = binconv0.ConvLayer[1]
conv1

Conv2d (64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [93]:
out2 = batch1(out1)
out3 = binact1(out2)
conv1.bias = None
out4 = conv1(out3)


In [96]:
out4

Variable containing:
( 0 , 0 ,.,.) = 
   -8  -26   -4  ...   -20  -22  -18
   -2  -18  -10  ...   -28  -22  -22
   -6  -24   -2  ...   -18  -16  -12
      ...         ⋱        ...      
    2  -10    8  ...    -2  -12   -8
   -2  -14    6  ...    -6  -14   -2
   10   10   18  ...    22    8   10

( 0 , 1 ,.,.) = 
   44   28   24  ...    36   22   16
   64   50   40  ...    58   40   36
   98   76   62  ...    62   56   42
      ...         ⋱        ...      
   90   66   64  ...    70   48   38
   62   36   58  ...    66   60   44
   54   46   48  ...    48   46   38

( 0 , 2 ,.,.) = 
    2    4   12  ...     4    4    6
   20   12   34  ...    20    8   20
   10    2   24  ...    12    2   16
      ...         ⋱        ...      
   22    4   18  ...     8    6   12
    4  -22  -16  ...   -24  -12    4
  -20  -34  -28  ...   -20  -30    8
    ... 

( 0 ,189,.,.) = 
   -6  -10   -2  ...    -6    2   -8
  -26  -20  -44  ...   -46  -34  -22
  -36  -26  -56  ...   -60  -50  -26
      ...  

In [98]:
quantizer.saved_params[1]


( 0 , 0 ,.,.) = 
  1.9854e-02  1.7430e-03  1.5917e-03 -1.2705e-02  1.4768e-02
  8.8550e-03  1.9816e-02 -1.2634e-02 -4.5918e-03 -1.1488e-02
  2.3038e-02 -2.1017e-02 -1.0588e-02  3.8625e-03  2.4222e-02
  2.4811e-02 -1.9536e-02  2.0398e-02 -2.3971e-04 -1.2513e-02
 -1.0656e-02  1.3622e-03 -6.4876e-03 -2.2427e-02 -3.1160e-03

( 0 , 1 ,.,.) = 
 -2.7056e-03 -4.3975e-03 -1.3461e-02 -1.5300e-02 -1.4789e-03
 -1.5433e-02 -2.3503e-02  9.6584e-03  2.3697e-02 -2.5940e-03
  4.8158e-04 -1.5426e-02 -5.9504e-03  2.0913e-02  3.2624e-03
 -1.6488e-04 -5.5602e-03 -6.5532e-03  1.3503e-02  1.3679e-02
  1.2023e-02  1.0051e-02 -1.0832e-03  9.2694e-03 -1.1804e-02

( 0 , 2 ,.,.) = 
 -8.8570e-03 -1.2342e-03  6.4216e-03 -2.3477e-02  1.5734e-04
 -1.3358e-02 -3.8932e-03  1.6753e-02  2.1799e-03 -1.8357e-02
 -2.2244e-02  3.7937e-03 -2.4062e-02 -8.4986e-03  1.2075e-02
  2.3265e-02 -1.9646e-02  2.0102e-02  1.2907e-02 -1.7970e-02
 -1.4858e-02 -3.2150e-03 -1.8726e-02 -1.7776e-03  2.7310e-03
    ... 

( 0 ,61 ,.,.) = 
 -7.

In [154]:
for layer in model.state_dict():
    print(layer)

first_feature.0.weight
first_feature.0.bias
last_layer.0.weight
last_layer.0.bias
last_layer.0.running_mean
last_layer.0.running_var
last_layer.2.weight
last_layer.2.bias
last_layer.3.weight
last_layer.3.bias
last_layer.3.running_mean
last_layer.3.running_var
features.1.ConvLayer.0.weight
features.1.ConvLayer.0.bias
features.1.ConvLayer.0.running_mean
features.1.ConvLayer.0.running_var
features.1.ConvLayer.2.weight
features.3.ConvLayer.0.weight
features.3.ConvLayer.0.bias
features.3.ConvLayer.0.running_mean
features.3.ConvLayer.0.running_var
features.3.ConvLayer.2.weight
features.3.ConvLayer.2.bias
features.4.ConvLayer.0.weight
features.4.ConvLayer.0.bias
features.4.ConvLayer.0.running_mean
features.4.ConvLayer.0.running_var
features.4.ConvLayer.2.weight
features.4.ConvLayer.2.bias
features.5.ConvLayer.0.weight
features.5.ConvLayer.0.bias
features.5.ConvLayer.0.running_mean
features.5.ConvLayer.0.running_var
features.5.ConvLayer.2.weight
features.5.ConvLayer.2.bias
features.7.weight
fe

In [196]:
t = torch.rand(5,4)
t


 0.2841  0.4114  0.9601  0.6169
 0.2486  0.3997  0.9347  0.5635
 0.0921  0.6981  0.7937  0.1600
 0.7425  0.5263  0.4888  0.2621
 0.8920  0.1198  0.2727  0.6142
[torch.FloatTensor of size 5x4]

In [197]:
s =t.size()
s

torch.Size([5, 4])

In [198]:
negMean = t.mean(1, keepdim=True).mul(-1).expand_as(t)
negMean


-0.5681 -0.5681 -0.5681 -0.5681
-0.5366 -0.5366 -0.5366 -0.5366
-0.4360 -0.4360 -0.4360 -0.4360
-0.5049 -0.5049 -0.5049 -0.5049
-0.4747 -0.4747 -0.4747 -0.4747
[torch.FloatTensor of size 5x4]

In [206]:
n = t.nelement()
n

20

In [204]:
t.norm(1,1,keepdim=True)


 2.2725
 2.1464
 1.7439
 2.0197
 1.8987
[torch.FloatTensor of size 5x1]

In [126]:
b = a.tolist()

In [127]:
type(b)

int

# computing Lut

In [9]:
tobin = lambda x, count=8: "".join(map(lambda y:str((x>>y)&1), range(count-1, -1, -1)))

In [30]:
def from_uint4_to_int16(x):
    b = ''
    a = tobin(x,4)
    for x in range(12):
        b = b+ a[0]
    b += a
    return b

from_uint4_to_int16(9)

'1111111111111001'

In [34]:
def twos_comp(binary_string, bits):
    val = int(binary_string,2)
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val      

In [37]:
c = ''
for i in range(256):
    n_low = i % 16
    n_low = from_uint4_to_int16(n_low)
    n_high = int(i / 16)
    n_high = from_uint4_to_int16(n_high)
    int16_val = twos_comp(n_high+n_low, 32)
    c += str(int16_val) + ','
    #print(i,n_low, n_high,int16_val)
print(c)

0,1,2,3,4,5,6,7,65528,65529,65530,65531,65532,65533,65534,65535,65536,65537,65538,65539,65540,65541,65542,65543,131064,131065,131066,131067,131068,131069,131070,131071,131072,131073,131074,131075,131076,131077,131078,131079,196600,196601,196602,196603,196604,196605,196606,196607,196608,196609,196610,196611,196612,196613,196614,196615,262136,262137,262138,262139,262140,262141,262142,262143,262144,262145,262146,262147,262148,262149,262150,262151,327672,327673,327674,327675,327676,327677,327678,327679,327680,327681,327682,327683,327684,327685,327686,327687,393208,393209,393210,393211,393212,393213,393214,393215,393216,393217,393218,393219,393220,393221,393222,393223,458744,458745,458746,458747,458748,458749,458750,458751,458752,458753,458754,458755,458756,458757,458758,458759,524280,524281,524282,524283,524284,524285,524286,524287,-524288,-524287,-524286,-524285,-524284,-524283,-524282,-524281,-458760,-458759,-458758,-458757,-458756,-458755,-458754,-458753,-458752,-458751,-458750,-458749,

In [28]:
b = ''
a = tobin(8,4)
for x in range(12):
    b = b+ a[0]
b += a
print(b)
[a[0] for x in range(8)]

1111111111111000


['1', '1', '1', '1', '1', '1', '1', '1']

'1111111111111000'

In [26]:
binary_string = '1001' # or whatever... no '0b' prefix
out = twos_comp(int(binary_string,2), len(binary_string))

In [23]:
model

mobilenet_quant_devel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d_SAME(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.003, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
      tensor([6.], requires_grad=True), inplace)
    )
    (1): Sequential(
      (0): Conv2d_SAME(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.003, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
      tensor([6.], requires_grad=True), inplace)
    )
    (2): Sequential(
      (0): Conv2d_SAME(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.003, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits

In [19]:
import mmdnn

In [20]:
from mmdnn.conversion.pytorch.pytorch_graph import PytorchGraph

In [53]:
g = PytorchGraph(model)

In [54]:
g.build([1,3,224,224])

TypeError: _jit_pass_onnx(): incompatible function arguments. The following argument types are supported:
    1. (arg0: torch::jit::Graph, arg1: torch._C._onnx.OperatorExportTypes) -> torch::jit::Graph

Invoked with: graph(%0 : Float(1, 3, 224, 224),
      %1 : Float(32, 3, 3, 3),
      %2 : Float(32),
      %3 : Float(32),
      %4 : Float(32),
      %5 : Float(32),
      %6 : Long(),
      %7 : Float(32, 1, 3, 3),
      %8 : Float(32),
      %9 : Float(32),
      %10 : Float(32),
      %11 : Float(32),
      %12 : Long(),
      %13 : Float(64, 32, 1, 1),
      %14 : Float(64),
      %15 : Float(64),
      %16 : Float(64),
      %17 : Float(64),
      %18 : Long(),
      %19 : Float(64, 1, 3, 3),
      %20 : Float(64),
      %21 : Float(64),
      %22 : Float(64),
      %23 : Float(64),
      %24 : Long(),
      %25 : Float(128, 64, 1, 1),
      %26 : Float(128),
      %27 : Float(128),
      %28 : Float(128),
      %29 : Float(128),
      %30 : Long(),
      %31 : Float(128, 1, 3, 3)

In [42]:
g.output_layers    

[]

In [17]:
model = models.__dict__[model_name]
model = model()


None None None
1.0 224


/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:209: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [21]:
model

mobilenet_real(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (4): Sequential(
      (0): Conv2d(64, 128, kern

In [28]:
for item in g.get_nodes():
    print(item)

In [32]:
g.shape_dict

{}

In [38]:
import torchvision.models as models

In [39]:
squeezenet = models.squeezenet1_0(pretrained=False)

In [65]:
squeezenet

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

In [69]:
g = PytorchGraph(model)
g.build([1,3,224,224])

RuntimeError: /pytorch/torch/csrc/jit/tracer.h:143: getTracingState: Assertion `var_state == state` failed.

In [43]:
dummy_input = Variable(torch.randn(1, 3, 224, 224))
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]
torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, output_names=output_names)

graph(%0 : Float(1, 3, 224, 224),
      %model.0.0.weight : Float(32, 3, 3, 3),
      %model.0.1.weight : Float(32),
      %model.0.1.bias : Float(32),
      %model.0.1.running_mean : Float(32),
      %model.0.1.running_var : Float(32),
      %model.0.1.num_batches_tracked : Long(),
      %model.1.0.weight : Float(32, 1, 3, 3),
      %model.1.1.weight : Float(32),
      %model.1.1.bias : Float(32),
      %model.1.1.running_mean : Float(32),
      %model.1.1.running_var : Float(32),
      %model.1.1.num_batches_tracked : Long(),
      %model.2.0.weight : Float(64, 32, 1, 1),
      %model.2.1.weight : Float(64),
      %model.2.1.bias : Float(64),
      %model.2.1.running_mean : Float(64),
      %model.2.1.running_var : Float(64),
      %model.2.1.num_batches_tracked : Long(),
      %model.3.0.weight : Float(64, 1, 3, 3),
      %model.3.1.weight : Float(64),
      %model.3.1.bias : Float(64),
      %model.3.1.running_mean : Float(64),
      %model.3.1.running_var : Float(64),
      %model

In [47]:
import onnx
modelonn = onnx.load("alexnet.onnx")

In [50]:
onnx.checker.check_model(modelonn)

In [52]:
print(onnx.helper.printable_graph(modelonn.graph))

graph torch-jit-export (
  %0[FLOAT, 1x3x224x224]
) initializers (
  %model.0.0.weight[FLOAT, 32x3x3x3]
  %model.0.1.weight[FLOAT, 32]
  %model.0.1.bias[FLOAT, 32]
  %model.0.1.running_mean[FLOAT, 32]
  %model.0.1.running_var[FLOAT, 32]
  %model.0.1.num_batches_tracked[INT64, scalar]
  %model.1.0.weight[FLOAT, 32x1x3x3]
  %model.1.1.weight[FLOAT, 32]
  %model.1.1.bias[FLOAT, 32]
  %model.1.1.running_mean[FLOAT, 32]
  %model.1.1.running_var[FLOAT, 32]
  %model.1.1.num_batches_tracked[INT64, scalar]
  %model.2.0.weight[FLOAT, 64x32x1x1]
  %model.2.1.weight[FLOAT, 64]
  %model.2.1.bias[FLOAT, 64]
  %model.2.1.running_mean[FLOAT, 64]
  %model.2.1.running_var[FLOAT, 64]
  %model.2.1.num_batches_tracked[INT64, scalar]
  %model.3.0.weight[FLOAT, 64x1x3x3]
  %model.3.1.weight[FLOAT, 64]
  %model.3.1.bias[FLOAT, 64]
  %model.3.1.running_mean[FLOAT, 64]
  %model.3.1.running_var[FLOAT, 64]
  %model.3.1.num_batches_tracked[INT64, scalar]
  %model.4.0.weight[FLOAT, 128x64x1x1]
  %model.4.1.weight[F

In [57]:
model.eval()
trace, output = torch.jit.get_trace_graph(model, (dummy_input, ))

In [60]:
trace.graph()

graph(%0 : Float(1, 3, 224, 224),
      %1 : Float(32, 3, 3, 3),
      %2 : Float(32),
      %3 : Float(32),
      %4 : Float(32),
      %5 : Float(32),
      %6 : Long(),
      %7 : Float(32, 1, 3, 3),
      %8 : Float(32),
      %9 : Float(32),
      %10 : Float(32),
      %11 : Float(32),
      %12 : Long(),
      %13 : Float(64, 32, 1, 1),
      %14 : Float(64),
      %15 : Float(64),
      %16 : Float(64),
      %17 : Float(64),
      %18 : Long(),
      %19 : Float(64, 1, 3, 3),
      %20 : Float(64),
      %21 : Float(64),
      %22 : Float(64),
      %23 : Float(64),
      %24 : Long(),
      %25 : Float(128, 64, 1, 1),
      %26 : Float(128),
      %27 : Float(128),
      %28 : Float(128),
      %29 : Float(128),
      %30 : Long(),
      %31 : Float(128, 1, 3, 3),
      %32 : Float(128),
      %33 : Float(128),
      %34 : Float(128),
      %35 : Float(128),
      %36 : Long(),
      %37 : Float(128, 128, 1, 1),
      %38 : Float(128),
      %39 : Float(128),
      %40 : Floa

In [62]:
list(trace.graph().nodes())

[%165 : Float(1, 3, 224, 224) = aten::clone(%0),
 %166 : Float(32, 3, 3, 3) = aten::clone(%1),
 %167 : Float(32) = aten::clone(%2),
 %168 : Float(32) = aten::clone(%3),
 %169 : Float(32) = aten::clone(%4),
 %170 : Float(32) = aten::clone(%5),
 %171 : Long() = aten::clone(%6),
 %172 : Float(32, 1, 3, 3) = aten::clone(%7),
 %173 : Float(32) = aten::clone(%8),
 %174 : Float(32) = aten::clone(%9),
 %175 : Float(32) = aten::clone(%10),
 %176 : Float(32) = aten::clone(%11),
 %177 : Long() = aten::clone(%12),
 %178 : Float(64, 32, 1, 1) = aten::clone(%13),
 %179 : Float(64) = aten::clone(%14),
 %180 : Float(64) = aten::clone(%15),
 %181 : Float(64) = aten::clone(%16),
 %182 : Float(64) = aten::clone(%17),
 %183 : Long() = aten::clone(%18),
 %184 : Float(64, 1, 3, 3) = aten::clone(%19),
 %185 : Float(64) = aten::clone(%20),
 %186 : Float(64) = aten::clone(%21),
 %187 : Float(64) = aten::clone(%22),
 %188 : Float(64) = aten::clone(%23),
 %189 : Long() = aten::clone(%24),
 %190 : Float(128, 64, 

In [94]:
from mmdnn.conversion.common.DataStructure.graph import GraphNode, Graph
import torch
import torch.jit
import torch.autograd
import torch.serialization
import contextlib
from torch.jit import _unique_state_dict



class PytorchGraphNode(GraphNode):

    def __init__(self, layer):
        self._name = layer.scopeName()
        self._kind = layer.kind()
        import re
        node_id = re.search(r"[\d]+", layer.__str__())
        self.id = node_id.group(0)

        super(PytorchGraphNode, self).__init__(layer)
        self.attrs = {k : layer[k] for k in layer.attributeNames()}

        self.weights_name = '.'.join(
            re.findall(r'\[([\w\d.]+)\]', self._name)
        )


    @property
    def name(self):
        name = self._name + self.id
        # Scopes created in a nested scope may have initial characters
        # that are illegal as the initial character of an op name
        # (viz. '-', '\', '/', and '_').
        name = name.replace('-','n').replace('\\','n').replace('/','n').replace('_','n').replace('[','n').replace(']','n')
        return name

    @property
    def type(self):
        return self._kind

    @property
    def pytorch_layer(self):
        return self.layer




class PytorchGraph(Graph):

    def __init__(self, model):
        # sanity check.
        super(PytorchGraph, self).__init__(model)
        self.model = model
        self.state_dict = _unique_state_dict(self.model)
        self.shape_dict = dict()


    @staticmethod
    def _optimize_graph(graph, aten, export_raw_ir=False):
        # run dce first to eliminate dead parts of the graph that might have been
        # left behind by things like symbolic_override

        torch._C._jit_pass_dce(graph)
        torch._C._jit_pass_lint(graph)

        torch._C._jit_pass_peephole(graph)
        torch._C._jit_pass_lint(graph)
        if not export_raw_ir:
            graph = torch._C._jit_pass_onnx(graph, aten)
 
            torch._C._jit_pass_lint(graph)
            torch._C._jit_pass_onnx_peephole(graph)
            torch._C._jit_pass_lint(graph)
        torch._C._jit_pass_dce(graph)
        torch._C._jit_pass_lint(graph)
        graph = torch._C._jit_pass_canonicalize(graph)
        torch._C._jit_pass_lint(graph)
        return graph


    @staticmethod
    def get_node_id(node):
        import re
        node_id = re.search(r"[\d]+", node.__str__())
        return node_id.group(0)

    @contextlib.contextmanager
    def set_training(self, model, mode):
        r"""
        A context manager to temporarily set the training mode of 'model'
        to 'mode', resetting it when we exit the with-block.  A no-op if
        mode is None.
        """
        if mode is None:
            yield
            return
        old_mode = model.training
        if old_mode != mode:
            model.train(mode)
        try:
            yield
        finally:
            if old_mode != mode:
                model.train(old_mode)


    def build(self, shape):
        """
        build graph for pytorch 0.4.0
        """

        import re
        # construct graph
        dummy_input = torch.autograd.Variable(torch.randn(shape), requires_grad=False)


        with self.set_training(self.model, False):
            trace, output = torch.jit.get_trace_graph(self.model, (dummy_input, ))

        #trace.set_graph(PytorchGraph._optimize_graph(trace.graph(), False))
        # nodes
        nodes = list(trace.graph().nodes())
        for node in nodes:
            if 'input' in str(node):

                print('*************************')
                print(node)
                print(node.scopeName())
                for i,node_input in enumerate(list(node.inputs())):
                    print(i, '\t',PytorchGraph.get_node_id(node_input.node()), node_input.node().scopeName())
                
                

        # input layer
        # TODO
        return True
        print('going to read stuff')

        # build each layer
        for node in nodes:
            node_id = PytorchGraph.get_node_id(node)
            node_scope = node.scopeName()
            node_name = node_scope + node_id
            print(node)
            print(node_name)
            node_name = node_name.replace('-','n').replace('\\','n').replace('/','n').replace('_','n').replace('[','n').replace(']','n')
            output_shape_str = re.findall(r'[^()!]+', node.__str__())[1]
            #output_shape = [int(x.replace('!', '')) for x in output_shape_str.split(',')]
            output_shape = [x.replace('!', '') for x in output_shape_str.split(',')]
            print(output_shape)


            self.shape_dict[node_name] = output_shape
            self.layer_map[node_name] = PytorchGraphNode(node)
            self.layer_name_map[node_name] = node_name

            # input
            for node_input in list(node.inputs()):

                if PytorchGraph.get_node_id(node_input.node()) and node_input.node().scopeName():
                    node_input_name = node_input.node().scopeName() + PytorchGraph.get_node_id(node_input.node())
                    node_input_name = node_input_name.replace('-','n').replace('\\','n').replace('/','n').replace('_','n').replace('[','n').replace(']','n')
                    print('Connection',node_input_name, node_name)
                    self._make_connection(node_input_name, node_name)
                    # print(node_input_name ,'->', node_name)
        print('Complete')

        super(PytorchGraph, self).build()

In [95]:
g = PytorchGraph(model)
g.build([1,3,224,224])

*************************
%input.1 : Float(1, 32, 112, 112) = aten::_convolution(%0, %1, %330, %333, %336, %339, %340, %343, %344, %345, %346, %347), scope: mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]

mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
0 	 0 
1 	 0 
2 	 330 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
3 	 333 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
4 	 336 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
5 	 339 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
6 	 340 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
7 	 343 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
8 	 344 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
9 	 345 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
10 	 346 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
11 	 347 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]
*************************
%input.2 : Float(1, 32, 112, 112) = 

True

In [67]:
model.graph()

AttributeError: 'mobilenet_real' object has no attribute 'graph'

In [23]:
import torch.utils.tensorboard

In [28]:
from collections import OrderedDict

In [30]:
f = OrderedDict(model.named_modules())['model.26.0']
f

Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [18]:
dummy_input = torch.autograd.Variable(torch.randn([1,3,224,224]), requires_grad=False)

In [25]:
g,v = torch.utils.tensorboard._pytorch_graph.graph(model,dummy_input)

In [40]:
OrderedDict(model.named_modules()).keys()

odict_keys(['', 'model', 'model.0', 'model.0.0', 'model.0.1', 'model.0.2', 'model.1', 'model.1.0', 'model.1.1', 'model.1.2', 'model.2', 'model.2.0', 'model.2.1', 'model.2.2', 'model.3', 'model.3.0', 'model.3.1', 'model.3.2', 'model.4', 'model.4.0', 'model.4.1', 'model.4.2', 'model.5', 'model.5.0', 'model.5.1', 'model.5.2', 'model.6', 'model.6.0', 'model.6.1', 'model.6.2', 'model.7', 'model.7.0', 'model.7.1', 'model.7.2', 'model.8', 'model.8.0', 'model.8.1', 'model.8.2', 'model.9', 'model.9.0', 'model.9.1', 'model.9.2', 'model.10', 'model.10.0', 'model.10.1', 'model.10.2', 'model.11', 'model.11.0', 'model.11.1', 'model.11.2', 'model.12', 'model.12.0', 'model.12.1', 'model.12.2', 'model.13', 'model.13.0', 'model.13.1', 'model.13.2', 'model.14', 'model.14.0', 'model.14.1', 'model.14.2', 'model.15', 'model.15.0', 'model.15.1', 'model.15.2', 'model.16', 'model.16.0', 'model.16.1', 'model.16.2', 'model.17', 'model.17.0', 'model.17.1', 'model.17.2', 'model.18', 'model.18.0', 'model.18.1', 'mo

In [94]:
def get_model_name(a):
    s = ''
    while a.find('[') > 0:
        s += a[a.find('[')+1:a.find(']')]+'.'
        a = a[a.find(']')+1:]
    return s[:-1]

def get_model_id(a, g):
    for i, node in enumerate(g.node):
        if node.name == a:
            return i
    return -1


ff = 'mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]/165'
v = get_model_name(ff)
v = get_model_id(ff, g)
print(v)


138


In [97]:
m = OrderedDict(model.named_modules())

for i,node in enumerate(g.node):
    if node.op in _IO_LIST:
        print('************')
        print(node.name)
        print(node.op)
    if node.op in _OP_LIST:
        print('************')
        v = get_model_name(node.name)
        print(i,get_model_id(node.name,g), m[v])
        print(node.name,node.op)
        for j, x in enumerate(node.input):
            id = get_model_id(x,g)
            print('Input',j,':',id, x, g.node[id].op, get_model_name(x))
            if g.node[id].op in _IO_LIST:
            elif g.node[id].op in _OP_LIST:
                




************
input/0
IO Node
************
138 138 Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]/165 onnx::Conv
0
Input 0 : 0 input/0 IO Node 
1
Input 1 : 1 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]/1 Parameter model.0.0
************
139 139 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
mobilenet_real/Sequential[model]/Sequential[0]/BatchNorm2d[1]/166 onnx::BatchNormalization
138
Input 0 : 138 mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]/165 onnx::Conv model.0.0
2
Input 1 : 2 mobilenet_real/Sequential[model]/Sequential[0]/BatchNorm2d[1]/2 Parameter model.0.1
3
Input 2 : 3 mobilenet_real/Sequential[model]/Sequential[0]/BatchNorm2d[1]/3 Parameter model.0.1
4
Input 3 : 4 mobilenet_real/Sequential[model]/Sequential[0]/BatchNorm2d[1]/4 Parameter model.0.1
5
Input 4 : 5 mobilenet_real/Sequential[model]/Sequential[0]/BatchNorm2d[1]/5 Parameter mod

Input 3 : 124 mobilenet_real/Sequential[model]/Sequential[24]/BatchNorm2d[1]/148 Parameter model.24.1
125
Input 4 : 125 mobilenet_real/Sequential[model]/Sequential[24]/BatchNorm2d[1]/149 Parameter model.24.1
************
212 212 ReLU6()
mobilenet_real/Sequential[model]/Sequential[24]/ReLU6[2]/239 onnx::Clip
211
Input 0 : 211 mobilenet_real/Sequential[model]/Sequential[24]/BatchNorm2d[1]/238 onnx::BatchNormalization model.24.1
************
213 213 Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=1024, bias=False)
mobilenet_real/Sequential[model]/Sequential[25]/Conv2d[0]/240 onnx::Conv
212
Input 0 : 212 mobilenet_real/Sequential[model]/Sequential[24]/ReLU6[2]/239 onnx::Clip model.24.2
126
Input 1 : 126 mobilenet_real/Sequential[model]/Sequential[25]/Conv2d[0]/151 Parameter model.25.0
************
214 214 BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
mobilenet_real/Sequential[model]/Sequential[25]/BatchNorm2d[1]/241 onnx::Bat

# new Quantop Part

In [136]:
m = OrderedDict(model.named_modules())
_OP_LIST = ['onnx::Gemm','onnx::BatchNormalization','onnx::Conv','onnx::Clip','onnx::AveragePool','onnx::Reshape']
_IO_LIST = ['IO Node']

quant_type =  'PerLayerAsymPACT'
weight_bits= 8
bias_bits = 8
batch_fold_type = 'folding_weights', 
batch_fold_delay = 0 
act_bits = 8
add_config = [] 


# generate quantized model
deployment_model = copy.deepcopy(model)
param_to_quantize = []
batch_fold = False


def has_children(module):
    try:
        next(module.children())
        return True
    except StopIteration:
        return False


        
def find_conv_node(node):
    ret = 0
    for j, x in enumerate(node.input):
        id = get_model_id(x,g)
        n = g.node[id]
        if n.op in _IO_LIST:
            pass
        elif n.op in _OP_LIST:
            if n.op in ['onnx::Gemm','onnx::Conv']:
                ret = id
            else:
                ret = find_conv_node(n)
            
    return ret

last_layer = None
modules_quant = [ ]           

replace_graph = {}
###################################################
##### Mapping the graph ###########################
###################################################
for i,node in enumerate(g.node):
    
    if node.op in _IO_LIST:
        print('************')
        print(node.name)
        print(node.op)
        
    if node.op in _OP_LIST:
        print('************')
        name = get_model_name(node.name)
        module = m[name]
        idx = get_model_id(node.name,g)
        print(i, idx, m[v])
        print(node.name,node.op)
        
        if node.op in ['onnx::Gemm','onnx::Conv']:
            layer_quant_descr = {}
            # saving per-node characteristics
            layer_quant_descr['w_bits'] = weight_bits
            layer_quant_descr['fold_type'] = batch_fold_type

            temp = getattr(submodel, 'quant_type', quant_type)
            if temp in _Availble_Quant_Type :
                layer_quant_descr['quant_type'] = temp
            else:
                print('Type of quantization not recognized')

            layer_quant_descr['conv'] = module
            layer_quant_descr['weight'] = module.weight.data.clone()
            layer_quant_descr['w_clip'] = None
            layer_quant_descr['id'] = idx
            layer_quant_descr['leaf'] = 0

            # import per-layer config from external config file
            idx_layer = len(param_to_quantize)
            for item_dict in add_config:
                if item_dict['layer'] == idx_layer:
                    if 'w_bits' in item_dict.keys():
                        layer_quant_descr['w_bits'] = item_dict['w_bits'] 

                    if 'fold_type' in item_dict.keys():
                        layer_quant_descr['fold_type'] = item_dict['fold_type'] 

                    if 'quant_type' in item_dict.keys():
                        layer_quant_descr['quant_type'] = item_dict['quant_type'] 

            if module.bias is None:
                layer_quant_descr['bias'] = False
            else:
                layer_quant_descr['bias'] = module.bias.data.clone()
            layer_quant_descr['bias_bits'] = bias_bits
            layer_quant_descr['batch_norm'] = None
            layer_quant_descr['act'] = None
            layer_quant_descr['quant_act'] = None

            # append into deployment graph
            quant_layer = copy.deepcopy(module)
            modules_quant.append(quant_layer)
            layer_quant_descr['quant_conv'] = quant_layer

            #PACT needs extra parameters for learning quantization range
            if layer_quant_descr['quant_type'] == 'PerLayerAsymPACT':
                layer_quant_descr['w_max_thr'] = nn.Parameter( module.weight.data.max().cuda(),requires_grad=True )
                layer_quant_descr['w_min_thr'] = nn.Parameter( module.weight.data.min().cuda(),requires_grad=True )
                layer_quant_descr['w_max_thr'].add(1).sum().backward() #fake to create grad
                layer_quant_descr['w_min_thr'].add(1).sum().backward() #fake to create grad

            elif layer_quant_descr['quant_type'] == 'PerLayerSymPACT':
                layer_quant_descr['w_max_thr'] = nn.Parameter( module.weight.data.max().cuda(),requires_grad=True )
                layer_quant_descr['w_max_thr'].add(1).sum().backward() #fake to create grad

            elif layer_quant_descr['quant_type'] == 'PerChannelsAsymMinMax':
                weight = module.weight.data
                max_weight,_ = weight.reshape(weight.size(0), -1).max(1)
                min_weight,_ = weight.reshape(weight.size(0), -1).min(1)
                layer_quant_descr['w_max_thr'] = nn.Parameter( max_weight.cuda(),requires_grad=True )
                layer_quant_descr['w_min_thr'] = nn.Parameter( min_weight.cuda(),requires_grad=True )
                layer_quant_descr['w_max_thr'].add(1).sum().backward() #fake to create grad
                layer_quant_descr['w_min_thr'].add(1).sum().backward() #fake to create grad
                layer_quant_descr['w_min_mat'] = None
                layer_quant_descr['w_max_mat'] = None
                if type(module) == nn.Linear:
                    print('PerLayerAsymPACT chennel with fixed batch')
                    out_ch = weight.size(0)
                    quant_act = ScaledClippedLinearQuantizationChannelBias(out_ch, clip_val=False, bias=False)
                    layer_quant_descr['quant_act'] = quant_act
                    modules_quant.append(quant_act)

            # append the latest node
            param_to_quantize.append(layer_quant_descr)
            replace_graph[name] = [module, module]

        elif node.op in ['onnx::BatchNormalization']:
            id_layer = find_conv_node(node)
            for item in param_to_quantize:
                if item['id'] == id_layer:
                    item['batch_norm'] = module
                    break
                    
            replace_graph[name] = [module, None]

        elif node.op in ['onnx::AveragePool']:
            modules_quant.append(module) # temporary - this should me merged into or previous layer
            replace_graph[name] = [module, module]

        elif node.op in ['onnx::Clip']:
            
            id_layer = find_conv_node(node)
            for item in param_to_quantize:
                if item['id'] == id_layer:
                    last_layer = item
                    break

            last_layer['act'] = module

            # quantized activations
            if type(module) in [ClippedLinearQuantization, LearnedClippedLinearQuantization, QuantActive, nn.ReLU6]:

                act_bits = 8
                #check if number of bits need to be changed
                for item_dict in add_config:
                    if item_dict['layer'] == idx_layer:
                        if 'a_bits' in item_dict.keys():
                            act_bits = item_dict['a_bits'] 
                            module.num_bits = act_bits                                 

                if last_layer['fold_type'] == 'folding_thresh':
                    quant_act = ScaledThresholdsQuantization4d()
                    quant_act.num_bits = act_bits
                    quant_act.n_thresholds = 2**act_bits -1

                elif last_layer['fold_type'] == 'ICN':
                    out_ch = last_layer['conv'].out_channels
                    quant_act = ScaledClippedLinearQuantizationChannel(out_ch,clip_val=2**act_bits -1)

                else:
                    quant_act = ScaledClippedLinearQuantization(clip_val=2**act_bits -1)

                last_layer['act_o_bits'] = act_bits

            else:
                print('Supported activation layer but no method is here yet!')


            replace_graph[name] = [LearnedClippedLinearQuantization(init_act_clip_val=6,num_bits=act_bits), quant_act]

            last_layer['quant_act'] = quant_act

            modules_quant.append(quant_act)
            
        else:
            print('Else name = ', name)
            replace_graph[name] = [module, module]



                
n_quantize_layers = len(param_to_quantize)

print('This is the quantized network: ')
print(deployment_model)
for i,item in enumerate(param_to_quantize):
    print('***** Layer', i,'**********')
    print(item['batch_norm'])
    print(item['id'])
    print(item['quant_conv'])
    print(item['quant_act'])

************
input/0
IO Node
************
138 138 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]/165 onnx::Conv
************
139 139 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[0]/BatchNorm2d[1]/166 onnx::BatchNormalization
************
140 140 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[0]/ReLU6[2]/167 onnx::Clip
************
141 141 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[1]/Conv2d[0]/168 onnx::Conv
************
142 142 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[1]/BatchNorm2d[1]/169 onnx::BatchNormalization
************
143 143 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[1]/ReLU6[2]/170 onnx::Clip
************
144 144 Linear(in_features=1024, o

204 204 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[22]/Conv2d[0]/231 onnx::Conv
************
205 205 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[22]/BatchNorm2d[1]/232 onnx::BatchNormalization
************
206 206 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[22]/ReLU6[2]/233 onnx::Clip
************
207 207 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[23]/Conv2d[0]/234 onnx::Conv
************
208 208 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[23]/BatchNorm2d[1]/235 onnx::BatchNormalization
************
209 209 Linear(in_features=1024, out_features=1000, bias=True)
mobilenet_real/Sequential[model]/Sequential[23]/ReLU6[2]/236 onnx::Clip
************
210 210 Linear(in_features=1024, out_features=1000, bias=True)
mobilen

In [137]:
replace_graph

{'': [mobilenet_real(
    (model): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6()
      )
      (1): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6()
      )
      (2): Sequential(
        (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6()
      )
      (3): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6()
      )
    

In [138]:
def has_children(module):
    try:
        next(module.children())
        return True
    except StopIteration:
        return False

In [145]:
fake_model = model
deployment_model = copy.deepcopy(model)

for name_sub,submodel in model.named_children():
    # map the sub-graph 
    modules_quant = [] 
    modules_fake = []
    for name, module in submodel.named_modules():
        if has_children(module) is False:
            if name is not '':
                n = name_sub+'.'+name
            else:
                n = name_sub
            v = replace_graph[n]
            fake = v[0]
            other = v[1]
            if fake is not None:
                modules_fake.append(fake)
            
            if other is not None:
                modules_quant.append(other)
            
    
    fake_model._modules[name_sub] = nn.Sequential(*modules_fake)
    deployment_model._modules[name_sub] = nn.Sequential(*modules_quant)


print(deployment_model)

KeyError: 'model.0'

In [135]:
replace_graph

[{'model.0.0': [Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
   Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)]},
 {'model.0.1': [BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   None]},
 {'model.0.2': [LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
   tensor([6.], requires_grad=True), inplace),
   ScaledClippedLinearQuantization(M0=1, N0=1, clip_val=255, inplace)]},
 {'model.1.0': [Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False),
   Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)]},
 {'model.1.1': [BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   None]},
 {'model.1.2': [LearnedClippedLinearQuantization(num_bits=8, clip_val=Parameter containing:
   tensor([6.], requires_grad=True), inplace),
   ScaledClippedLinearQuantization(M0=1, N0=1, clip_val

In [120]:


for name_sub,submodel in model.named_children():
    # map the sub-graph 
    for name, module in submodel.named_modules():
        if has_children(module) is False:
            print(module)
                    



Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU6()
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU6()
Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU6()
Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU6()
Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU6()
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU6(

In [118]:
list(*model)

TypeError: type object argument after * must be an iterable, not mobilenet_real

In [101]:
from models.quantized_modules import ABC_MxNBinConv, ABC_binarize, Quantize, QuantActive, LinQuantActive, BinActive
from models.linear_quantized_modules import ClippedLinearQuantization, LearnedClippedLinearQuantization, ScaledClippedLinearQuantization, ScaledThresholdsQuantization4d, \
                                            Conv2d_SAME, ScaledClippedLinearQuantizationChannel, ScaledClippedLinearQuantizationChannelBias

_Availble_Quant_Type = ['BNN', 'XNORNET', 'PerLayerAsymMinMax', 'PerLayerSymPACT', 'PerLayerAsymPACT', 'PerChannelsAsymMinMax', None]
_Supported_Activ_Funct = [nn.ReLU6, nn.ReLU, ClippedLinearQuantization, LearnedClippedLinearQuantization, BinActive]



quant_type =  'PerLayerAsymPACT'
weight_bits= 8
bias_bits = 8
batch_fold_type = 'folding_weights', 
batch_fold_delay = 0 
act_bits = 8
add_config = [] 


# generate quantized model
deployment_model = copy.deepcopy(model)
param_to_quantize = []
batch_fold = False

print('Batch Folding: ', batch_fold, 'Batch Folding Delay: ', batch_fold_delay, 'Type', batch_fold_type )

#start
last_layer = None
modules_quant = [ ]

print(add_config)



for name_sub,submodel in model.named_children():

    # map the sub-graph 
    for name, module in submodel.named_modules():
        if has_children(module) is False:

            layer_quant_descr = {}

            if type(module) in [nn.Conv2d, nn.Linear, Conv2d_SAME]:

                # saving per-node characteristics
                layer_quant_descr['name'] = name
                layer_quant_descr['w_bits'] = weight_bits
                layer_quant_descr['fold_type'] = batch_fold_type

                temp = getattr(submodel, 'quant_type', quant_type)
                if temp in _Availble_Quant_Type :
                    layer_quant_descr['quant_type'] = temp
                else:
                    print('Type of quantization not recognized')

                layer_quant_descr['conv'] = module
                layer_quant_descr['weight'] = module.weight.data.clone()
                layer_quant_descr['w_clip'] = None

                # import per-layer config from external config file
                idx_layer = len(param_to_quantize)
                for item_dict in add_config:
                    if item_dict['layer'] == idx_layer:
                        if 'w_bits' in item_dict.keys():
                            layer_quant_descr['w_bits'] = item_dict['w_bits'] 

                        if 'fold_type' in item_dict.keys():
                            layer_quant_descr['fold_type'] = item_dict['fold_type'] 

                        if 'quant_type' in item_dict.keys():
                            layer_quant_descr['quant_type'] = item_dict['quant_type'] 

                if module.bias is None:
                    layer_quant_descr['bias'] = False
                else:
                    layer_quant_descr['bias'] = module.bias.data.clone()
                layer_quant_descr['bias_bits'] = bias_bits
                layer_quant_descr['batch_norm'] = None
                layer_quant_descr['act'] = None
                layer_quant_descr['quant_act'] = None

                # append into deployment graph
                quant_layer = copy.deepcopy(module)
                modules_quant.append(quant_layer)
                layer_quant_descr['quant_conv'] = quant_layer

                #PACT needs extra parameters for learning quantization range
                if layer_quant_descr['quant_type'] == 'PerLayerAsymPACT':
                    layer_quant_descr['w_max_thr'] = nn.Parameter( module.weight.data.max().cuda(),requires_grad=True )
                    layer_quant_descr['w_min_thr'] = nn.Parameter( module.weight.data.min().cuda(),requires_grad=True )
                    layer_quant_descr['w_max_thr'].add(1).sum().backward() #fake to create grad
                    layer_quant_descr['w_min_thr'].add(1).sum().backward() #fake to create grad

                elif layer_quant_descr['quant_type'] == 'PerLayerSymPACT':
                    layer_quant_descr['w_max_thr'] = nn.Parameter( module.weight.data.max().cuda(),requires_grad=True )
                    layer_quant_descr['w_max_thr'].add(1).sum().backward() #fake to create grad

                elif layer_quant_descr['quant_type'] == 'PerChannelsAsymMinMax':
                    weight = module.weight.data
                    max_weight,_ = weight.reshape(weight.size(0), -1).max(1)
                    min_weight,_ = weight.reshape(weight.size(0), -1).min(1)
                    layer_quant_descr['w_max_thr'] = nn.Parameter( max_weight.cuda(),requires_grad=True )
                    layer_quant_descr['w_min_thr'] = nn.Parameter( min_weight.cuda(),requires_grad=True )
                    layer_quant_descr['w_max_thr'].add(1).sum().backward() #fake to create grad
                    layer_quant_descr['w_min_thr'].add(1).sum().backward() #fake to create grad
                    layer_quant_descr['w_min_mat'] = None
                    layer_quant_descr['w_max_mat'] = None
                    if type(module) == nn.Linear:
                        print('PerLayerAsymPACT chennel with fixed batch')
                        out_ch = weight.size(0)
                        quant_act = ScaledClippedLinearQuantizationChannelBias(out_ch, clip_val=False, bias=False)
                        layer_quant_descr['quant_act'] = quant_act
                        modules_quant.append(quant_act)

                # append the latest node
                param_to_quantize.append(layer_quant_descr)
                last_layer = param_to_quantize[-1]

            elif (type(module) in [nn.BatchNorm2d]):
                if last_layer is not None:
                    last_layer['batch_norm'] = module

            elif type(module) in [nn.AvgPool2d]:
                modules_quant.append(module) # temporary - this should me merged into or previous layer

            elif type(module) in _Supported_Activ_Funct:

                # check if a quantized features an activation function
                if last_layer is not None:

                    last_layer['act'] = module

                    # quantized activations
                    if type(module) in [ClippedLinearQuantization, LearnedClippedLinearQuantization, QuantActive]:

                        act_bits = module.num_bits
                        #check if number of bits need to be changed
                        for item_dict in add_config:
                            if item_dict['layer'] == idx_layer:
                                if 'a_bits' in item_dict.keys():
                                    act_bits = item_dict['a_bits'] 
                                    module.num_bits = act_bits                                 

                        if last_layer['fold_type'] == 'folding_thresh':
                            quant_act = ScaledThresholdsQuantization4d()
                            quant_act.num_bits = act_bits
                            quant_act.n_thresholds = 2**act_bits -1

                        elif last_layer['fold_type'] == 'ICN':
                            out_ch = last_layer['conv'].out_channels
                            quant_act = ScaledClippedLinearQuantizationChannel(out_ch,clip_val=2**act_bits -1)

                        else:
                            quant_act = ScaledClippedLinearQuantization(clip_val=2**act_bits -1)

                        last_layer['act_o_bits'] = act_bits

                    # full-precision activations
                    elif  type(module) is nn.ReLU6:
                        quant_act = nn.ReLU6(inplace=True)

                    # binary acivation
                    elif  type(module) is BinActive :
                        act_bits = 1
                        quant_act = BinActive()

                    else:
                        print('Supported activation layer but no method is here yet!')

                        

                    last_layer['quant_act'] = quant_act

                    modules_quant.append(quant_act)

            else:
                print(type(module), 'not supported yet!')

        else:
            #if type(module) is not nn.Sequential:
            last_layer = None

    deployment_model._modules[name_sub] = nn.Sequential(*modules_quant)
    modules_quant = [  ] 


n_quantize_layers = len(param_to_quantize)

print('This is the quantized network: ')
print(deployment_model)



Batch Folding:  False Batch Folding Delay:  0 Type ('folding_weights',)
[]


NameError: name 'has_children' is not defined

In [27]:
for item,v in model.named_modules():
    print(item)


model
model.0
model.0.0
model.0.1
model.0.2
model.1
model.1.0
model.1.1
model.1.2
model.2
model.2.0
model.2.1
model.2.2
model.3
model.3.0
model.3.1
model.3.2
model.4
model.4.0
model.4.1
model.4.2
model.5
model.5.0
model.5.1
model.5.2
model.6
model.6.0
model.6.1
model.6.2
model.7
model.7.0
model.7.1
model.7.2
model.8
model.8.0
model.8.1
model.8.2
model.9
model.9.0
model.9.1
model.9.2
model.10
model.10.0
model.10.1
model.10.2
model.11
model.11.0
model.11.1
model.11.2
model.12
model.12.0
model.12.1
model.12.2
model.13
model.13.0
model.13.1
model.13.2
model.14
model.14.0
model.14.1
model.14.2
model.15
model.15.0
model.15.1
model.15.2
model.16
model.16.0
model.16.1
model.16.2
model.17
model.17.0
model.17.1
model.17.2
model.18
model.18.0
model.18.1
model.18.2
model.19
model.19.0
model.19.1
model.19.2
model.20
model.20.0
model.20.1
model.20.2
model.21
model.21.0
model.21.1
model.21.2
model.22
model.22.0
model.22.1
model.22.2
model.23
model.23.0
model.23.1
model.23.2
model.24
model.24.0
model

In [29]:
list(*model)

TypeError: type object argument after * must be an iterable, not mobilenet_real

In [31]:
ff = 'mobilenet_real/Sequential[model]/Sequential[0]/Conv2d[0]/1'

In [182]:
ff.find('[')

25

In [130]:
for item in model.modules():
    print(item.named_children)

<bound method Module.named_children of mobilenet_real(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (4): Sequ

In [134]:
mobilenet_real/Sequential[model]/Sequential[24]/Conv2d[0]/237

SyntaxError: invalid syntax (<ipython-input-134-0615a994443e>, line 1)

In [32]:
torch.__version__

'0.4.1'